
# Set up the notebook

In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

from choropleth_utils import ChoroplethUtilities
from stats_scraping_utils import StatsScrapingUtilities
from storage import Storage
import pandas as pd
import re
import os
import numpy as np

s = Storage()
ssu = StatsScrapingUtilities(s=s)

In [52]:

column_description_dict = s.load_object('column_description_dict')
us_stats_df = s.load_object('us_stats_df')

all_countries_df = s.load_object('all_countries_df').set_index('country_code', drop=True)
all_countries_df.country_name = all_countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
s.store_objects(all_countries_df=all_countries_df.reset_index(drop=False))

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\all_countries_df.pkl



----
# Get State/Country Cost of Living Equivalents


## Clean and prepare US States dataset

In [39]:

url = 'https://meric.mo.gov/data/cost-living-data-series'
tables_list = ssu.get_page_tables(url)

[(1, (6, 8)), (0, (0, 9))]


In [40]:

body_opener_str = '</thead>\n  <tbody>\n  '
head_closer_str = '<th>Misc.</th>\n    </tr>\n    '
html_str = tables_list[0].to_html().replace(body_opener_str, '').replace(head_closer_str,
                                                                         head_closer_str + body_opener_str).replace('th>', 'td>')
tables_list = ssu.get_page_tables(html_str)

[(0, (52, 10))]


In [41]:

us_states_df = tables_list[0].copy()
# print(us_states_df.columns.tolist())
floats_list = ['cost_of_living_index', 'grocery_index', 'housing_index', 'utilities_index', 'transportation_index',
                'health_index', 'misc_index']
columns_list = ['cost_of_living_rank', 'state_name'] + floats_list
us_states_df.columns = ['deleteme'] + columns_list
us_states_df = us_states_df[columns_list]
for cn in floats_list:
    us_states_df[cn] = us_states_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x)))
    us_states_df[cn] = pd.to_numeric(us_states_df[cn], errors='coerce', downcast='float')
mask_series = us_states_df.state_name.isnull()
us_states_df = us_states_df[~mask_series]
us_states_df.head(10).T

,0,1,2,3,4,5,6,7,8,9
cost_of_living_rank,1,2,3,4,5,6,7,8,9,10
state_name,Mississippi,Oklahoma,Kansas,Alabama,Iowa,Georgia,Indiana,Tennessee,Arkansas,Michigan
cost_of_living_index,83.099998,84.800003,85.400002,87.5,88.099998,88.199997,89.199997,89.400002,89.900002,89.900002
grocery_index,92.300003,94.0,91.699997,97.0,99.400002,94.099998,93.400002,94.5,92.900002,90.400002
housing_index,63.799999,68.800003,68.199997,68.900002,69.5,74.199997,75.900002,80.699997,76.699997,77.800003
utilities_index,90.0,95.199997,98.699997,101.099998,94.800003,91.199997,100.800003,94.099998,96.5,98.199997
transportation_index,89.099998,90.300003,94.199997,90.0,93.400002,91.400002,97.099998,87.900002,94.5,95.699997
health_index,97.400002,92.900002,102.800003,90.5,99.099998,94.300003,95.400002,91.699997,80.099998,97.0
misc_index,92.0,89.699997,90.400002,95.5,95.599998,96.699997,93.800003,93.800003,99.699997,96.599998


In [42]:

# Remove DC from the comparison so it doesn't skew the results
mask_series = (us_states_df.state_name.isin(['District of Columbia', '***']))
us_states_df = us_states_df[~mask_series]

In [43]:

# Remove US states duplicates and misspellings
states_list = sorted(set(us_states_df.state_name).symmetric_difference(set(ssu.us_stats_df.index)))
print(states_list)
doubles_df = ssu.check_4_doubles(states_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = us_states_df.duplicated(subset=['state_name'], keep=False)
if us_states_df[mask_series].shape[0]:
    display(us_states_df[mask_series])

['District of Columbia']



## Clean and prepare Countries dataset

In [45]:

url = 'https://www.numbeo.com/cost-of-living/rankings_by_country.jsp'
tables_list = ssu.get_page_tables(url)

[(1, (137, 8)), (0, (1, 2))]


In [50]:

countries_df = tables_list[1].copy()
# print(countries_df.columns.tolist())
floats_list = ['cost_of_living_index', 'rent_index', 'cost_of_living_plus_rent_index', 'groceries Index',
               'restaurant_price_index', 'local_purchasing_power_index']
columns_list = ['country_name'] + floats_list
countries_df.columns = ['cost_of_living_rank'] + columns_list
for cn in floats_list:
    countries_df[cn] = countries_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x)))
    countries_df[cn] = pd.to_numeric(countries_df[cn], errors='coerce', downcast='float')
mask_series = countries_df.country_name.isnull()
countries_df = countries_df[~mask_series][columns_list]
# countries_df.country_name = countries_df.country_name.map(lambda x: str(x).split('*')[0].strip())
countries_df.head(9).T

,0,1,2,3,4,5,6,7,8
country_name,Bermuda,Switzerland,Bahamas,Barbados,Iceland,Norway,Jersey,Singapore,Israel
cost_of_living_index,141.740005,110.339996,88.269997,87.07,86.589996,85.93,80.379997,79.089996,77.279999
rent_index,98.959999,50.209999,38.41,21.16,35.810001,30.549999,55.369999,65.089996,32.950001
cost_of_living_plus_rent_index,121.389999,81.730003,64.550003,55.709999,62.43,59.59,68.480003,72.43,56.189999
groceries Index,142.619995,113.349998,71.919998,81.730003,82.440002,81.309998,64.519997,71.360001,66.449997
restaurant_price_index,144.740005,104.300003,94.290001,74.5,88.639999,87.589996,89.230003,52.200001,83.919998
local_purchasing_power_index,82.339996,116.190002,44.220001,32.619999,82.620003,86.269997,68.720001,94.650002,73.540001


In [53]:

# Remove country duplicates and misspellings
countries_df.country_name = countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
countries_list = sorted(set(countries_df.country_name).symmetric_difference(set(all_countries_df.country_name)))
if countries_list:
    print(countries_list)
doubles_df = ssu.check_4_doubles(countries_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = countries_df.duplicated(subset=['country_name'], keep=False)
if countries_df[mask_series].shape[0]:
    display(countries_df[mask_series])

['Afghanistan', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua & Barbuda', 'Aruba', 'Benin', 'Bhutan', 'Bonaire, Sint Eustatius & Saba', 'Bouvet Island', 'British Indian Ocean Territory', 'British Virgin Islands', 'Burkina Faso', 'Burundi', 'Cayman Islands', 'Central African Republic', 'Chad', 'Christmas Island', 'Cocos (Keeling) Islands', 'Comoros', 'Cook Islands', 'Curaçao', "Côte d'Ivoire", 'DRC', 'Djibouti', 'Dominica', 'Equatorial Guinea', 'Eritrea', 'Eswatini', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Federated States of Micronesia', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Gibraltar', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guinea', 'Guinea-Bissau', 'Haiti', 'Heard Island and McDonald Islands', 'Holy See', 'Isle of Man', 'Ivory Coast', 'Kiribati', 'Kosovo (Disputed Territory)', 'Laos', 'Lesotho', 'Liberia', 'Liechtenstein', 'Madagascar', 'Malawi', 'Mali', 'Marshall Islands', 'Martinique', 'M

,first_item,second_item,max_similarity
38,Gambia,Zambia,0.833333
58,Malawi,Mali,0.800000
13,British Virgin Islands,US Virgin Islands,0.769231
22,Cook Islands,Norfolk Island,0.769231
40,Greenland,Grenada,0.750000
16,Cayman Islands,Åland Islands,0.740741
84,Sint Maarten,St. Martin,0.727273
85,Solomon Islands,Åland Islands,0.714286
3,Angola,Anguilla,0.714286
80,San Marino,St. Martin,0.700000


In [56]:

# Normalize the index so that the US is 100
mask_series = (countries_df.country_name == 'USA')
cost_of_living_index = countries_df[mask_series].cost_of_living_index.squeeze()
countries_df['normalized_cost_of_living_index'] = countries_df.cost_of_living_index.map(lambda x: 100*x/cost_of_living_index)
mask_series = (countries_df.country_name == 'USA')
countries_df[mask_series].T

,14
country_name,USA
cost_of_living_index,69.919998
rent_index,46.860001
cost_of_living_plus_rent_index,58.950001
groceries Index,70.059998
restaurant_price_index,65.75
local_purchasing_power_index,99.879997
normalized_cost_of_living_index,100.0



## Prepare for and Create Choropleth

In [57]:

equivalence_column_name = 'Country_Equivalent_Cost_of_Living'
states_target_column_name = 'cost_of_living_index'
mask_series = countries_df.country_name.isin(all_countries_df.country_name)
ssu.prepare_for_choroplething(countries_df[mask_series], 'normalized_cost_of_living_index', us_states_df,
                              st_col_name=states_target_column_name,
                              st_col_explanation='Cost of Living Index (2022)',
                              equivalence_column_name=equivalence_column_name, verbose=True)


Albania (48.05) is close to the normalized cost of living index of Mississippi (83.10)
Algeria (35.87) is close to the normalized cost of living index of Mississippi (83.10)
Argentina (45.95) is close to the normalized cost of living index of Mississippi (83.10)
Armenia (55.72) is close to the normalized cost of living index of Mississippi (83.10)
Australia (103.36) is close to the normalized cost of living index of Nevada (103.10)
Austria (91.69) is close to the normalized cost of living index of Illinois (91.70)
Azerbaijan (40.52) is close to the normalized cost of living index of Mississippi (83.10)
Bahamas (126.24) is close to the normalized cost of living index of Maryland (126.40)
Bahrain (77) is close to the normalized cost of living index of Mississippi (83.10)
Bangladesh (41.88) is close to the normalized cost of living index of Mississippi (83.10)
Barbados (124.53) is close to the normalized cost of living index of Oregon (125.20)
Belarus (37.74) is close to the normalized c

In [58]:

c = ChoroplethUtilities(iso_3166_2_code='us', one_country_df=ssu.us_stats_df, all_countries_df=all_countries_df)
c.create_label_line_file()
svg_file_path = c.create_country_colored_labeled_map(
    numeric_column_name=states_target_column_name, string_column_name=equivalence_column_name,
    one_country_df=ssu.us_stats_df, cmap='summer')
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\US_cost_of_living_index_Country_Equivalent_Cost_of_Living.svg



----
# Get State/Country Income Inequality Equivalents


## Clean and prepare US States dataset

In [ ]:

url = 'https://worldpopulationreview.com/state-rankings/income-inequality-by-state'
tables_list = ssu.get_page_tables(url)

In [5]:

us_states_df = tables_list[0].copy()
# print(us_states_df.columns.tolist())
us_states_df.columns = ['state_name', 'gini_coefficient']
for cn in ['gini_coefficient']:
    us_states_df[cn] = us_states_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x)))
    us_states_df[cn] = pd.to_numeric(us_states_df[cn], errors='coerce', downcast='float')
mask_series = us_states_df.state_name.isnull()
us_states_df = us_states_df[~mask_series]
us_states_df.sample(5).T

,25,4,0,40,23
state_name,West Virginia,Florida,New York,Nebraska,Ohio
gini_coefficient,46.209999,49.0,51.02,44.200001,46.41


In [6]:

mask_series = (us_states_df.state_name == 'District of Columbia')
us_states_df = us_states_df[~mask_series]

In [7]:

# Remove US states duplicates and misspellings
states_list = sorted(set(us_states_df.state_name).symmetric_difference(set(ssu.us_stats_df.index)))
print(states_list)
doubles_df = ssu.check_4_doubles(states_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = us_states_df.duplicated(subset=['state_name'], keep=False)
if us_states_df[mask_series].shape[0]:
    display(us_states_df[mask_series])

['District of Columbia']


In [8]:

s.store_objects(income_inequality_us_states_df=us_states_df)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\income_inequality_us_states_df.pkl



## Clean and prepare Countries dataset

In [9]:

if s.pickle_exists('income_inequality_countries_df'):
    countries_df = s.load_object('income_inequality_countries_df')
else:
    driver = ssu.get_driver()
    tables_list = ssu.get_page_tables('https://worldpopulationreview.com/country-rankings/wealth-inequality-by-country', driver=driver)
    driver.close()
    countries_df = tables_list[0].copy()
    # print(countries_df.columns.tolist())
    countries_df.columns = ['country_name', 'gini_index', 'country_population_2022']
    for cn in ['gini_index']:
        countries_df[cn] = pd.to_numeric(countries_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x))), errors='coerce', downcast='float')
    mask_series = countries_df.country_name.isnull()
    countries_df = countries_df[~mask_series]
countries_df.head(9).T

,0,1,2,3,4,5,6,7,8
country_name,South Africa,Namibia,Suriname,Zambia,São Tomé & Príncipe,Central African Republic,Eswatini,Mozambique,Brazil
gini_index,63.0,59.099998,57.900002,57.099998,56.299999,56.200001,54.599998,54.0,53.400002
country_population_2022,59893885,2567012,618040,20017675,227380,5579144,1201670,32969518,215313498


In [10]:

# Remove country duplicates and misspellings
countries_df.country_name = countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
countries_list = sorted(set(countries_df.country_name).symmetric_difference(set(all_countries_df.country_name)))
doubles_df = ssu.check_4_doubles(countries_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = countries_df.duplicated(subset=['country_name'], keep=False)
if countries_df[mask_series].shape[0]:
    display(countries_df[mask_series])

,first_item,second_item,max_similarity
14,British Virgin Islands,US Virgin Islands,0.769231
20,Cook Islands,Norfolk Island,0.769231
34,Greenland,Grenada,0.750000
17,Cayman Islands,Åland Islands,0.740741
72,Sint Maarten,St. Martin,0.727273
38,Guernsey,Jersey,0.714286
69,San Marino,St. Martin,0.700000
12,Bouvet Island,Faroe Islands,0.692308
28,Faroe Islands,Åland Islands,0.692308
6,Aruba,Cuba,0.666667


In [11]:

s.store_objects(income_inequality_countries_df=countries_df)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\income_inequality_countries_df.pkl



## Prepare for and Create Choropleth

In [ ]:

equivalence_column_name = 'Country_Equivalent_Income_Inequality'
states_target_column_name = 'gini_coefficient'
# mask_series = countries_df.country_name.isin(all_countries_df.country_name)
mask_series = countries_df.country_name.isin(ssu.oecd_countries_list)
ssu.prepare_for_choroplething(countries_df[mask_series], 'gini_index', us_states_df, st_col_name=states_target_column_name,
                              st_col_explanation='Gini Coefficient (2022)',
                              equivalence_column_name=equivalence_column_name, verbose=True)

In [50]:

c = ChoroplethUtilities(iso_3166_2_code='us', one_country_df=ssu.us_stats_df, all_countries_df=all_countries_df)
c.create_label_line_file()
svg_file_path = c.create_country_colored_labeled_map(
    numeric_column_name=states_target_column_name, string_column_name=equivalence_column_name,
    one_country_df=ssu.us_stats_df, cmap='summer')
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\US_gini_coefficient_Country_Equivalent_Income_Inequality.svg



----
# Get State/Country CO<sub>2</sub> Emissions Equivalents


## Clean and prepare US States dataset

In [21]:

url = 'https://www.bts.gov/browse-statistical-products-and-data/state-transportation-statistics/energy-consumption-and-co2'
tables_list = ssu.get_page_tables(url)

No tables found
[]


In [23]:

us_states_df = s.load_csv('Energy_Consumption_and_CO2_Emissions_by_us_state')
# print(us_states_df.columns.tolist())
us_states_df.columns = ['state_name', 'measure_str', 'sector_str', 'measure_year', 'pivot_list', 'co2_emissions_mmt']
mask_series = us_states_df.state_name.isin(ssu.us_states_list) & (us_states_df.sector_str == 'Total') & (us_states_df.measure_year == 2017)
print(us_states_df[mask_series].shape)
columns_list = ['state_name', 'co2_emissions_mmt']
us_states_df = us_states_df[mask_series][columns_list]
print(us_states_df.co2_emissions_mmt.sum())

url = 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/national/totals/nst-est2019-alldata.csv'
POPULATION_DICT = pd.read_csv(url, encoding=s.encoding_type).set_index('NAME').POPESTIMATE2017.to_dict()
def f(row_series):
    state_name = row_series.state_name
    co2_emissions_mmt = row_series.co2_emissions_mmt
    
    return 1_000*co2_emissions_mmt/POPULATION_DICT[state_name]
us_states_df['co2_emissions_mmt_per_capita'] = us_states_df.apply(f, axis='columns')

us_states_df.sample(5)

(51, 6)
5166.4


,state_name,co2_emissions_mmt,co2_emissions_mmt_per_capita
1758,Maryland,51.9,0.008616
162,Alaska,34.3,0.046370
2094,Mississippi,67.8,0.022687
3438,South Carolina,69.2,0.013781
1926,Michigan,152.7,0.015311


In [25]:

mask_series = (us_states_df.state_name == 'District of Columbia')
us_states_df = us_states_df[~mask_series]

In [26]:

# Remove US states duplicates and misspellings
states_list = sorted(set(us_states_df.state_name).symmetric_difference(set(ssu.us_stats_df.index)))
doubles_df = ssu.check_4_doubles(states_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = us_states_df.duplicated(subset=['state_name'], keep=False)
if us_states_df[mask_series].shape[0]:
    display(us_states_df[mask_series])


## Clean and prepare Countries dataset

In [28]:

driver = ssu.get_driver()
tables_list = ssu.get_page_tables('https://worldpopulationreview.com/country-rankings/carbon-footprint-by-country', driver=driver)
driver.close()

Getting the FireFox driver
[(0, (210, 6))]


In [29]:

countries_df = tables_list[0].copy()
# print(countries_df.columns.tolist())
countries_df.columns = ['country_name', 'co2_emissions_mt_2020',  'co2_emissions_mt_2017',
                        'co2_emissions_per_capita_2020', 'co2_emissions_per_capita_2017', 'country_population_2022']
for cn in ['co2_emissions_mt_2020', 'co2_emissions_mt_2017', 'co2_emissions_per_capita_2020', 'co2_emissions_per_capita_2017']:
    countries_df[cn] = pd.to_numeric(countries_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x))), errors='coerce', downcast='float')
mask_series = countries_df.country_name.isnull()
countries_df = countries_df[~mask_series]
countries_df.country_name = countries_df.country_name.map(lambda x: str(x).split('*')[0].strip())
countries_df.head(9).T

,0,1,2,3,4,5,6,7,8
country_name,China,United States,India,Russia,Japan,Iran,Germany,South Korea,Saudi Arabia
co2_emissions_mt_2020,11680.419922,4535.299805,2411.72998,1674.22998,1061.77002,690.23999,636.880005,621.469971,588.809998
co2_emissions_mt_2017,10877.219727,5107.390137,2454.77002,1764.869995,1320.780029,671.450012,796.530029,673.320007,638.76001
co2_emissions_per_capita_2020,8.2,13.68,1.74,11.64,8.39,8.26,7.72,12.07,16.959999
co2_emissions_per_capita_2017,7.7,15.7,1.8,12.3,10.4,8.3,9.7,13.2,19.4
country_population_2022,1425887337,338289857,1417173173,144713314,123951692,88550570,83369843,51815810,36408820


In [30]:

# Remove country duplicates and misspellings
countries_df.country_name = countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
countries_list = sorted(set(countries_df.country_name).symmetric_difference(set(all_countries_df.country_name)))
doubles_df = ssu.check_4_doubles(countries_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = countries_df.duplicated(subset=['country_name'], keep=False)
if countries_df[mask_series].shape[0]:
    display(countries_df[mask_series])

,first_item,second_item,max_similarity
10,Falkland Islands,Åland Islands,0.827586
32,Sint Maarten,St. Martin,0.727273
15,Guernsey,Jersey,0.714286
21,Marshall Islands,Åland Islands,0.620690



## Prepare for and Create Choropleth

In [ ]:

equivalence_column_name = 'Country_Equivalent_CO2_Emissions'
states_target_column_name = 'co2_emissions_mmt'
mask_series = countries_df.country_name.isin(all_countries_df.country_name)
ssu.prepare_for_choroplething(countries_df[mask_series], 'co2_emissions_mt_2017', us_states_df, st_col_name=states_target_column_name,
                              st_col_explanation='CO2 Emissions in Millions of Metric Tons (2017)',
                              equivalence_column_name=equivalence_column_name, verbose=True)

In [32]:

c = ChoroplethUtilities(iso_3166_2_code='us', one_country_df=ssu.us_stats_df, all_countries_df=all_countries_df)
c.create_label_line_file()
svg_file_path = c.create_country_colored_labeled_map(
    numeric_column_name=states_target_column_name, string_column_name=equivalence_column_name,
    one_country_df=ssu.us_stats_df, cmap='summer')
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\US_co2_emissions_mmt_Country_Equivalent_CO2_Emissions.svg



----
# Get State/Country Suicide Rate Equivalents


## Clean and prepare US States dataset

In [4]:

url = 'https://www.cdc.gov/nchs/pressroom/sosmap/suicide-mortality/suicide.htm'
tables_list = ssu.get_page_tables(url)
if not tables_list:
    tables_list = [pd.read_csv('../data/csv/suicide_2019.csv', encoding=s.encoding_type)]

No tables found
[]


In [5]:

us_states_df = tables_list[0].copy()
print(us_states_df.columns.tolist())
us_states_df.columns = ['source_year', 'state_abbreviation', 'suicide_rate', 'body_count', 'source_url']
for cn in ['source_year', 'suicide_rate', 'body_count']:
    us_states_df[cn] = us_states_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x)))
    us_states_df[cn] = pd.to_numeric(us_states_df[cn], errors='coerce', downcast='integer')
abbreviation_dict = {v: k for k, v in ssu.us_states_abbreviation_dict.items()}
mask_series = us_states_df.state_abbreviation.isnull()
us_states_df = us_states_df[~mask_series]
us_states_df['state_name'] = us_states_df.state_abbreviation.map(lambda x: abbreviation_dict.get(x, x))
mask_series = (us_states_df.source_year == 2019)
us_states_df = us_states_df[mask_series]
us_states_df.sample(5).T

['YEAR', 'STATE', 'RATE', 'DEATHS', 'URL']


,82,71,54,67,57
source_year,2019,2019,2019,2019,2019
state_abbreviation,NC,MI,CA,LA,DE
suicide_rate,12.5,14.3,10.7,15.0,11.3
body_count,1358,1472,4436,704,111
source_url,https://www.cdc.gov/nchs/pressroom/states/nort...,https://www.cdc.gov/nchs/pressroom/states/mich...,https://www.cdc.gov/nchs/pressroom/states/cali...,https://www.cdc.gov/nchs/pressroom/states/loui...,https://www.cdc.gov/nchs/pressroom/states/dela...
state_name,North Carolina,Michigan,California,Louisiana,Delaware


In [6]:

# Remove US states duplicates and misspellings
states_list = sorted(set(us_states_df.state_name).symmetric_difference(set(ssu.us_stats_df.index)))
print(states_list)
doubles_df = ssu.check_4_doubles(states_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = us_states_df.duplicated(subset=['state_name'], keep=False)
if us_states_df[mask_series].shape[0]:
    display(us_states_df[mask_series])

['District of Columbia']



## Clean and prepare Countries dataset

In [7]:

url = 'https://en.wikipedia.org/wiki/List_of_countries_by_suicide_rate'
tables_list = ssu.get_page_tables(url)

[(1, (190, 4)), (2, (184, 21)), (3, (184, 21)), (4, (184, 21)), (6, (112, 5)), (7, (12, 2)), (0, (10, 1)), (9, (8, 2)), (8, (6, 2)), (5, (5, 5))]


In [8]:

countries_df = tables_list[1].copy()
print(countries_df.columns.tolist())
columns_list = ['suicide_rate', 'male_suicide_rate', 'female_suicide_rate']
countries_df.columns = ['country_name'] + columns_list
for cn in columns_list:
    countries_df[cn] = countries_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x)))
    countries_df[cn] = pd.to_numeric(countries_df[cn], errors='coerce', downcast='float')
mask_series = countries_df.country_name.isnull()
countries_df = countries_df[~mask_series]
countries_df.country_name = countries_df.country_name.map(lambda x: str(x).split('*')[0].strip())
countries_df.head(9).T

['Country', 'All', 'Male', 'Female']


,0,1,2,3,4,5,6,7,8
country_name,Afghanistan,Albania,Algeria,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria
suicide_rate,6.0,3.7,2.6,12.6,0.3,8.1,2.7,11.3,10.4
male_suicide_rate,6.2,5.3,3.3,21.700001,0.0,13.5,4.9,17.0,16.6
female_suicide_rate,5.7,2.2,1.9,4.7,0.6,3.3,1.0,5.6,4.6


In [9]:

# Remove country duplicates and misspellings
countries_df.country_name = countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
countries_list = sorted(set(countries_df.country_name).symmetric_difference(set(all_countries_df.country_name)))
if countries_list:
    print(countries_list)
doubles_df = ssu.check_4_doubles(countries_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = countries_df.duplicated(subset=['country_name'], keep=False)
if countries_df[mask_series].shape[0]:
    display(countries_df[mask_series])

['Africa', 'American Samoa', 'Americas', 'Andorra', 'Anguilla', 'Antarctica', 'Aruba', 'Bermuda', 'Bonaire, Sint Eustatius & Saba', 'Bouvet Island', 'British Indian Ocean Territory', 'British Virgin Islands', 'Cape Verde', 'Cayman Islands', 'Christmas Island', 'Cocos (Keeling) Islands', 'Cook Islands', 'Curaçao', "Côte d'Ivoire", 'Dominica', 'East Timor', 'Eastern Mediterranean', 'Europe', 'Falkland Islands (Malvinas)', 'Faroe Islands', 'Federated States of Micronesia', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gibraltar', 'Global', 'Greenland', 'Guadeloupe', 'Guam', 'Guernsey', 'Heard Island and McDonald Islands', 'Holy See', 'Hong Kong', 'Isle of Man', 'Ivory Coast', 'Jersey', 'Liechtenstein', 'Macau', 'Marshall Islands', 'Martinique', 'Mayotte', 'Micronesia', 'Monaco', 'Montserrat', 'Nauru', 'New Caledonia', 'Niue', 'Norfolk Island', 'Northern Mariana Islands', 'Palau', 'Palestine', 'Pitcairn', 'Puerto Rico', 'ROC', 'Réunion', 'Saint Helena, Ascension & Tr

,first_item,second_item,max_similarity
11,British Virgin Islands,US Virgin Islands,0.769231
16,Cook Islands,Norfolk Island,0.769231
13,Cayman Islands,Åland Islands,0.740741
62,Sint Maarten,St. Martin,0.727273
0,Africa,Americas,0.714286
34,Guernsey,Jersey,0.714286
61,San Marino,St. Martin,0.700000
9,Bouvet Island,Faroe Islands,0.692308
24,Faroe Islands,Åland Islands,0.692308
1,American Samoa,Americas,0.636364



## Prepare for and Create Choropleth

In [ ]:

equivalence_column_name = 'Country_Equivalent_Suicide_Rate'
states_target_column_name = 'suicide_rate'
mask_series = countries_df.country_name.isin(all_countries_df.country_name)
ssu.prepare_for_choroplething(countries_df[mask_series], 'suicide_rate', us_states_df, st_col_name=states_target_column_name,
                              st_col_explanation='Suicide Rate per 100,000 Population (2019)',
                              equivalence_column_name=equivalence_column_name, verbose=False)

In [12]:

c = ChoroplethUtilities(iso_3166_2_code='us', one_country_df=ssu.us_stats_df, all_countries_df=all_countries_df)
c.create_label_line_file()
svg_file_path = c.create_country_colored_labeled_map(numeric_column_name=states_target_column_name,
                                                     string_column_name=equivalence_column_name,
                                                     one_country_df=ssu.us_stats_df, cmap='winter')
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\US_suicide_rate_Country_Equivalent_Suicice_Rate.svg



----
# Get State/Country Prison Population Equivalents


## Clean and prepare US States dataset

In [4]:

url = 'https://worldpopulationreview.com/state-rankings/prison-population-by-state'
tables_list = ssu.get_page_tables(url)

[(0, (50, 4))]


In [5]:

us_states_df = tables_list[0].copy()
print(us_states_df.columns.tolist())
columns_list = ['imprisonment_rate_per_100k', 'total_prison_population', 'state_population_2022']
us_states_df.columns = ['state_name'] + columns_list
for cn in columns_list:
    us_states_df[cn] = us_states_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x)))
    us_states_df[cn] = pd.to_numeric(us_states_df[cn], errors='coerce', downcast='integer')
mask_series = us_states_df.state_name.isnull()
us_states_df = us_states_df[~mask_series]
us_states_df.sample(5).T

['State', 'Imprisonment Rate (per 100K)', 'Total Prison Population', '2022 Pop.']


,38,37,2,11,0
state_name,Alaska,Connecticut,Oklahoma,Wyoming,Louisiana
imprisonment_rate_per_100k,241,242,633,428,674
total_prison_population,1782,8751,25338,2479,31584
state_population_2022,738023,3612314,4000953,579495,4682633


In [6]:

# Remove US states duplicates and misspellings
states_list = sorted(set(us_states_df.state_name).symmetric_difference(set(ssu.us_stats_df.index)))
doubles_df = ssu.check_4_doubles(states_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = us_states_df.duplicated(subset=['state_name'], keep=False)
if us_states_df[mask_series].shape[0]:
    display(us_states_df[mask_series])


## Clean and prepare Countries dataset

In [7]:

url = 'https://worldpopulationreview.com/country-rankings/incarceration-rates-by-country'
tables_list = ssu.get_page_tables(url)

[(0, (220, 5))]


In [8]:

countries_df = tables_list[0].copy()
print(countries_df.columns.tolist())
columns_list = ['incarceration_rate',  'total_prison_population', 'male_prison_population_percent', 'female_prison_population_percent']
countries_df.columns = ['country_name'] + columns_list
for cn in columns_list:
    countries_df[cn] = countries_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x).split('%')[0]))
    countries_df[cn] = pd.to_numeric(countries_df[cn], errors='coerce', downcast='integer')
mask_series = countries_df.country_name.isnull()
countries_df = countries_df[~mask_series]
countries_df.country_name = countries_df.country_name.map(lambda x: str(x).split('*')[0].strip())
countries_df.head(9).T

['Country', 'Incarceration Rate', 'Total Incarcerated', '% Male', '% Female']


,0,1,2,3,4,5,6,7,8
country_name,United States,Rwanda,Turkmenistan,El Salvador,Cuba,Palau,British Virgin Islands,Thailand,Panama
incarceration_rate,629.0,580.0,576.0,564.0,510.0,478.0,477.0,445.0,434.0
total_prison_population,2068800.0,76099.0,35000.0,36663.0,57337.0,86.0,143.0,309282.0,18942.0
male_prison_population_percent,90,95,94,93,0,95,96,89,95
female_prison_population_percent,10,5,7,7,0,5,5,12,5


In [9]:

# Remove country duplicates and misspellings
countries_df.country_name = countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
countries_list = sorted(set(countries_df.country_name).symmetric_difference(set(all_countries_df.country_name)))
if countries_list:
    print(countries_list)
doubles_df = ssu.check_4_doubles(countries_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = countries_df.duplicated(subset=['country_name'], keep=False)
if countries_df[mask_series].shape[0]:
    display(countries_df[mask_series])

['Antarctica', 'Bonaire, Sint Eustatius & Saba', 'Bouvet Island', 'British Indian Ocean Territory', 'Cape Verde', 'Christmas Island', 'Cocos (Keeling) Islands', "Côte d'Ivoire", 'Falkland Islands (Malvinas)', 'Federated States of Micronesia', 'French Southern Territories', 'Heard Island and McDonald Islands', 'Holy See', 'Ivory Coast', 'Micronesia', 'Montserrat', 'Niue', 'Norfolk Island', 'Palestine', 'Pitcairn', 'ROC', 'Réunion', 'Saint Helena, Ascension & Tristan da Cunha', 'South Georgia and the South Sandwich Islands', 'St. Barthélemy', 'St. Martin', 'St. Pierre & Miquelon', 'Svalbard and Jan Mayen', 'Tokelau', 'Turks & Caicos Islands', 'UK', 'United States Minor Outlying Islands', 'Wallis & Futuna', 'Western Sahara', 'Åland Islands']



## Prepare for and Create Choropleth

In [ ]:

equivalence_column_name = 'Country_Equivalent_Prison_Population'
states_target_column_name = 'total_prison_population'
mask_series = countries_df.country_name.isin(all_countries_df.country_name)
ssu.prepare_for_choroplething(countries_df[mask_series], 'total_prison_population', us_states_df, st_col_name=states_target_column_name,
                              st_col_explanation='Total Incarcerated (2020)',
                              equivalence_column_name=equivalence_column_name, verbose=True)

In [12]:

c = ChoroplethUtilities(iso_3166_2_code='us', one_country_df=ssu.us_stats_df, all_countries_df=all_countries_df)
c.create_label_line_file()
svg_file_path = c.create_country_colored_labeled_map(numeric_column_name=states_target_column_name,
                                                     string_column_name=equivalence_column_name,
                                                     one_country_df=ssu.us_stats_df)
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\US_total_prison_population_Country_Equivalent_Prison_Population.svg



----
# Get State/Country Intentional Homicide Equivalents


## Clean and prepare US States dataset

In [4]:

url = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_intentional_homicide_rate'
tables_list = ssu.get_page_tables(url)

[(0, (54, 12)), (1, (11, 2))]


In [5]:

us_states_df = tables_list[0].copy()
# print(us_states_df.columns.tolist())
columns_list = ['murder_victums_2020'] + [f'murder_rates_per_100k_people_{i}' for i in range(2020, 2010, -1)]
us_states_df.columns = ['state_name'] + columns_list
for cn in columns_list:
    us_states_df[cn] = us_states_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x)))
    us_states_df[cn] = pd.to_numeric(us_states_df[cn], errors='coerce', downcast='float')
mask_series = us_states_df.state_name.isnull()
us_states_df = us_states_df[~mask_series]
us_states_df.sample(5).T

,30,44,29,37,52
state_name,Wisconsin,Wyoming,California,Washington,New Hampshire
murder_victums_2020,308.0,18.0,2203.0,301.0,12.0
murder_rates_per_100k_people_2020,5.3,3.1,5.6,3.9,0.9
murder_rates_per_100k_people_2019,3.2,2.2,4.3,2.7,2.4
murder_rates_per_100k_people_2018,3.0,2.4,4.4,3.1,1.6
murder_rates_per_100k_people_2017,3.3,2.4,4.6,3.0,1.0
murder_rates_per_100k_people_2016,4.0,3.4,4.9,2.9,1.4
murder_rates_per_100k_people_2015,4.2,2.7,4.8,2.5,1.1
murder_rates_per_100k_people_2014,2.8,2.7,4.4,2.3,1.2
murder_rates_per_100k_people_2013,2.8,2.9,4.6,3.1,1.7


In [6]:

mask_series = (us_states_df.state_name == 'District of Columbia')
us_states_df = us_states_df[~mask_series]

In [7]:

# Remove US states duplicates and misspellings
states_list = sorted(set(us_states_df.state_name).symmetric_difference(set(ssu.us_stats_df.index)))
doubles_df = ssu.check_4_doubles(states_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = us_states_df.duplicated(subset=['state_name'], keep=False)
if us_states_df[mask_series].shape[0]:
    display(us_states_df[mask_series])


## Clean and prepare Countries dataset

In [8]:

url = 'https://en.wikipedia.org/wiki/List_of_countries_by_intentional_homicide_rate'
tables_list = ssu.get_page_tables(url)

[(1, (196, 7)), (2, (49, 2)), (3, (16, 2)), (0, (6, 3)), (4, (6, 2))]


In [9]:

countries_df = tables_list[1].copy()
# print(countries_df.columns.tolist())
countries_df.columns = ['country_name', 'region_name',  'subregion_name', 'homicide_rate',
                        'body_count', 'data_source_year', 'data_source']
for cn in ['homicide_rate', 'body_count', 'data_source_year']:
    countries_df[cn] = countries_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x).split('[')[0]))
    countries_df[cn] = pd.to_numeric(countries_df[cn], errors='coerce', downcast='integer')
mask_series = countries_df.country_name.isnull()
countries_df = countries_df[~mask_series]
countries_df.country_name = countries_df.country_name.map(lambda x: str(x).split('*')[0].strip())
countries_df.head(9).T

,1,2,3,4,5,6,7,8,9
country_name,Afghanistan,Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia
region_name,Asia,Europe,Africa,Europe,Africa,Americas,Americas,Americas,Asia
subregion_name,Southern Asia,Southern Europe,Northern Africa,Southern Europe,Middle Africa,Caribbean,Caribbean,South America,Western Asia
homicide_rate,6.7,2.1,1.3,2.6,4.8,28.3,11.1,5.3,1.8
body_count,2474.0,61.0,580.0,2.0,1217.0,4.0,10.0,2362.0,52.0
data_source_year,2018.0,2020.0,2020.0,2020.0,2012.0,2014.0,2012.0,2018.0,2020.0
data_source,NSO,NSO,CTS,CTS,NSO,SDG,OAS,MoS,CTS


In [10]:

# Remove country duplicates and misspellings
countries_df.country_name = countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
countries_list = sorted(set(countries_df.country_name).symmetric_difference(set(all_countries_df.country_name)))
doubles_df = ssu.check_4_doubles(countries_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = countries_df.duplicated(subset=['country_name'], keep=False)
if countries_df[mask_series].shape[0]:
    display(countries_df[mask_series])

,first_item,second_item,max_similarity
11,Cook Islands,Norfolk Island,0.769231
7,Channel Islands,Faroe Islands,0.714286
28,Guernsey,Jersey,0.714286
57,Solomon Islands,Åland Islands,0.714286
47,Northern Ireland,Northern Mariana Islands,0.700000
3,Bouvet Island,Faroe Islands,0.692308
20,Faroe Islands,Åland Islands,0.692308
46,North Korea,Northern Ireland,0.666667
63,Togo,Tonga,0.666667
9,Cocos (Keeling) Islands,Cook Islands,0.628571



## Prepare for and Create Choropleth

In [11]:

equivalence_column_name = 'Country_Equivalent_Intentional_Homicide_Rate'
states_target_column_name = 'murder_rates_per_100k_people_2020'
mask_series = countries_df.country_name.isin(all_countries_df.country_name)
ssu.prepare_for_choroplething(countries_df[mask_series], 'homicide_rate', us_states_df, st_col_name=states_target_column_name,
                              st_col_explanation='Intentional Homicides per 100,000 Population (2020)',
                              equivalence_column_name=equivalence_column_name, verbose=False)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\us_stats_df.pkl
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\column_description_dict.pkl


In [20]:

c = ChoroplethUtilities(iso_3166_2_code='us', one_country_df=ssu.us_stats_df, all_countries_df=all_countries_df)
c.create_label_line_file()
svg_file_path = c.create_country_colored_labeled_map(numeric_column_name=states_target_column_name,
                                                     string_column_name=equivalence_column_name,
                                                     one_country_df=ssu.us_stats_df, cmap='summer')
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\US_murder_rates_per_100k_people_2020_Country_Equivalent_Intentional_Homicide_Rate.svg



----
# Get State/Country Car Fatality Equivalents


## Clean and prepare US States dataset

In [4]:

# url = 'https://worldpopulationreview.com/state-rankings/fatal-car-accidents-by-state'
url = 'https://crashstats.nhtsa.dot.gov/Api/Public/Publication/812581'
file_name = '2016_State_Traffic_Data_CrashStats_NHTSA.pdf'
tables_list = ssu.get_page_tables(url, pdf_file_name=file_name)

Got stderr: Aug 17, 2022 6:14:27 PM org.apache.pdfbox.pdfparser.COSParser parseXref



[(3, (56, 9)), (7, (56, 6)), (6, (55, 3)), (8, (55, 7)), (9, (55, 7)), (10, (55, 5)), (4, (54, 9)), (5, (54, 7)), (2, (46, 3)), (0, (26, 19)), (11, (21, 2)), (1, (5, 4))]


In [5]:

us_states_df = tables_list[3].copy()
# columns_list = []
# for t1, t2, t3, t4 in zip(us_states_df.columns.tolist(), us_states_df.iloc[0].T.tolist(), us_states_df.iloc[1].T.tolist(),
#                           us_states_df.iloc[2].T.tolist()):
#     column_names_list = []
#     if not t1.startswith('Unnamed'):
#         column_names_list.append(t1.strip().lower().replace(' ', '_').replace(',000', 'k'))
#     for t in [t2, t3, t4]:
#         if str(t) != 'nan':
#             column_names_list.append(t.strip().lower().replace(' ', '_').replace(',000', 'k'))
#     columns_list.append('_'.join(column_names_list))
# print(columns_list)
columns_list = ['traffic_fatalities', 'population_in_thousands', 'licensed_drivers_in_thousands',
                'registered_vehicles_in_thousands', 'vehicle_miles_traveled_in_millions', 'fatality_rates_per_100k_population']
us_states_df.columns = ['state_name'] + columns_list + ['fixme', 'fatality_rates_per_100_million_vehicle_miles_traveled']
us_states_df = us_states_df.iloc[3:]
fixme_columns_list = ['fatality_rates_per_100k_licensed_drivers', 'fatality_rates_per_100k_registered_vehicles']
for i in range(2):
    us_states_df[fixme_columns_list[i]] = us_states_df.fixme.map(lambda x: re.sub(r'[^0-9\.]+', '', x.split(' ')[i]))
for cn in fixme_columns_list:
    us_states_df[cn] = pd.to_numeric(us_states_df[cn], errors='coerce', downcast='float')
columns_list += ['fatality_rates_per_100_million_vehicle_miles_traveled']
for cn in columns_list:
    us_states_df[cn] = us_states_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x)))
    us_states_df[cn] = pd.to_numeric(us_states_df[cn], errors='coerce', downcast='float')
columns_list = ['state_name'] + columns_list + fixme_columns_list
us_states_df[columns_list].head(5).T

,3,4,5,6,7
state_name,Alabama,Alaska,Arizona,Arkansas,California
Traffic Fatalities,1038.0,84.0,962.0,545.0,3623.0
Population (thousands),4863.0,742.0,6931.0,2988.0,39250.0
Licensed Drivers (thousands),3943.0,535.0,5082.0,2391.0,26199.0
Registered Vehicles (thousands),5468.0,795.0,5787.0,2808.0,30221.0
Vehicle Miles Traveled (millions),69227.0,5259.0,65786.0,35755.0,340115.0
fatality_rates_per_100k_population,21.34,11.32,13.88,18.24,9.23
Fatality Rates per 100 Million Vehicle Miles Traveled,1.5,1.6,1.46,1.52,1.07
"Fatality Rates per 100,000 Licensed Drivers",26.32,15.71,18.93,22.790001,13.83
"Fatality Rates per 100,000 Registered Vehicles",18.98,10.57,16.620001,19.41,11.99


In [6]:

# Remove US states duplicates and misspellings
states_list = sorted(set(us_states_df.state_name).symmetric_difference(set(ssu.us_stats_df.index)))
doubles_df = ssu.check_4_doubles(states_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False)

,first_item,second_item,max_similarity



## Clean and prepare Countries dataset

In [7]:

url = 'https://en.wikipedia.org/wiki/List_of_countries_by_traffic-related_death_rate'
tables_list = ssu.get_page_tables(url)

[(1, (189, 7)), (2, (4, 2)), (0, (1, 2))]


In [8]:

countries_df = tables_list[1].copy()
# print(countries_df.columns.tolist())
countries_df.columns = ['country_name', 'continent_name',  'road_deaths_per_100k_inhabitants',
                        'road_deaths_per_100k_motor_vehicles', 'road_deaths_per_100B_vehicle_kms', 'road_deaths_total',
                        'data_source_year']
for cn in ['road_deaths_per_100k_inhabitants', 'road_deaths_per_100k_motor_vehicles', 'road_deaths_per_100B_vehicle_kms',
           'road_deaths_total', 'data_source_year']:
    countries_df[cn] = countries_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x).split('[')[0]))
    countries_df[cn] = pd.to_numeric(countries_df[cn], errors='coerce', downcast='float')
countries_df.head(9).T

,0,1,2,3,4,5,6,7,8
country_name,Global,Africa,Eastern Mediterranean,Western Pacific,South-East Asia,Americas,Europe,Afghanistan,Albania
continent_name,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Asia,Europe
road_deaths_per_100k_inhabitants,18.200001,26.6,18.0,16.9,20.700001,15.6,9.3,15.5,15.1
road_deaths_per_100k_motor_vehicles,NaN,574.0,139.0,69.0,101.0,33.0,19.0,722.400024,107.199997
road_deaths_per_100B_vehicle_kms,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
road_deaths_total,1350000.0,246719.0,122730.0,328591.0,316080.0,153789.0,85629.0,4734.0,478.0
data_source_year,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2016.0,2013.0,2016.0


In [9]:

mask_series = (countries_df.country_name == 'Iceland')
countries_df[mask_series].T

,76
country_name,Iceland
continent_name,Europe
road_deaths_per_100k_inhabitants,3.8
road_deaths_per_100k_motor_vehicles,7.6
road_deaths_per_100B_vehicle_kms,4.9
road_deaths_total,8.0
data_source_year,5.20162


In [10]:

# Remove country duplicates and misspellings
countries_df.country_name = countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
countries_list = sorted(set(countries_df.country_name).symmetric_difference(set(all_countries_df.country_name)))
doubles_df = ssu.check_4_doubles(countries_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False)

,first_item,second_item,max_similarity
12,British Virgin Islands,US Virgin Islands,0.769231
35,Greenland,Grenada,0.750000
16,Cayman Islands,Åland Islands,0.740741
65,Sint Maarten,St. Martin,0.727273
0,Africa,Americas,0.714286
39,Guernsey,Jersey,0.714286
10,Bouvet Island,Faroe Islands,0.692308
29,Faroe Islands,Åland Islands,0.692308
1,American Samoa,Americas,0.636364
30,French Guiana,French Polynesia,0.620690


In [11]:

mask_series = countries_df.duplicated(subset=['country_name'], keep=False)
countries_df[mask_series].T

""
country_name
continent_name
road_deaths_per_100k_inhabitants
road_deaths_per_100k_motor_vehicles
road_deaths_per_100B_vehicle_kms
road_deaths_total
data_source_year



## Prepare for and Create Choropleth

In [12]:

equivalence_column_name = 'Country_Equivalent_Road_Deaths_per_100k_Inhabitants'
states_target_column_name = 'fatality_rates_per_100k_population'
mask_series = countries_df.country_name.isin(all_countries_df.country_name)
ssu.prepare_for_choroplething(countries_df[mask_series], 'road_deaths_per_100k_inhabitants', us_states_df, st_col_name=states_target_column_name,
                              st_col_explanation='Road Deaths per 100,000 Population',
                              equivalence_column_name=equivalence_column_name, verbose=False)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\us_stats_df.pkl
Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\column_description_dict.pkl


In [13]:

c = ChoroplethUtilities(iso_3166_2_code='us', one_country_df=ssu.us_stats_df, all_countries_df=all_countries_df)
c.create_label_line_file()
svg_file_path = c.create_country_colored_labeled_map(numeric_column_name=states_target_column_name,
                                                     string_column_name=equivalence_column_name,
                                                     one_country_df=ssu.us_stats_df)
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\US_fatality_rates_per_100k_population_Country_Equivalent_Road_Deaths_per_100k_Inhabitants.svg



----
# Get State/Country Physicians per Capita Equivalents


## Clean and prepare US States dataset

In [4]:

url = 'https://www.beckershospitalreview.com/workforce/this-state-has-the-most-physicians-per-capita.html'
tables_list = ssu.get_page_tables(url)

[(0, (52, 4))]


In [5]:

us_states_df = tables_list[0].copy()
states_target_column_name = 'physicians_per_10k'
us_states_df.columns = ['state_name', 'total_population', 'total_active_physicians', states_target_column_name]
us_states_df = us_states_df.iloc[1:]
us_states_df[states_target_column_name] = pd.to_numeric(us_states_df[states_target_column_name].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x))),
                                                        errors='coerce', downcast='float')
us_states_df[states_target_column_name] = us_states_df[states_target_column_name].map(lambda x: x/10)
us_states_df.sample(5)

,state_name,total_population,total_active_physicians,physicians_per_10k
22,Alaska,731545,2101,287.2
1,District of Columbia,705749,6147,87.1
32,Arizona,7278717,18343,25.2
2,Massachusetts,6892503,32116,46.6
15,Ohio,11689100,35333,302.3


In [14]:

# Why does MA only have 46.6 physicians per 10K whilst NY has 389.4?
mask_series = us_states_df.state_name.isin(['Massachusetts', 'New York'])
us_states_df[mask_series]

,state_name,total_population,total_active_physicians,physicians_per_10k
2,Massachusetts,6892503,32116,46.6
4,New York,19453561,75749,389.4


In [6]:

# Remove US states duplicates and misspellings
states_list = sorted(set(us_states_df.state_name).symmetric_difference(set(us_stats_df.index)))
doubles_df = ssu.check_4_doubles(states_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False)

,first_item,second_item,max_similarity



## Clean and prepare Countries dataset

In [7]:

tables_list = ssu.get_page_tables('https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_number_of_physicians', driver=None)

[(1, (188, 5)), (0, (1, 2))]


In [8]:

countries_df = tables_list[1].copy()
# print(countries_df.columns.tolist())
countries_target_column_name = 'physicians_per_10k'
LATEST_COLUMN_NAME = countries_target_column_name + '_latest'
EARLIER_COLUMN_NAME = countries_target_column_name + '_2013'
EARLIEST_COLUMN_NAME = countries_target_column_name + '_2009'
countries_df.columns = ['country_name', 'total_population', EARLIEST_COLUMN_NAME, EARLIER_COLUMN_NAME, LATEST_COLUMN_NAME]
for cn in [EARLIEST_COLUMN_NAME, EARLIER_COLUMN_NAME, LATEST_COLUMN_NAME]:
    countries_df[cn] = pd.to_numeric(countries_df[cn].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x))), errors='coerce', downcast='float')
def f(row_series):
    latest = row_series[LATEST_COLUMN_NAME]
    earlier = row_series[EARLIER_COLUMN_NAME]
    earliest = row_series[EARLIEST_COLUMN_NAME]
    for cv in [latest, earlier, earliest]:
        if str(cv) != 'nan':

            return cv
countries_df[countries_target_column_name] = countries_df.apply(f, axis='columns')
countries_df.head(5)

,country_name,total_population,physicians_per_10k_2009,physicians_per_10k_2013,physicians_per_10k_latest,physicians_per_10k
0,Australia,19612,100.0,327.0,NaN,327.0
1,Austria,31175,380.0,483.0,NaN,483.0
2,Azerbaijan,32388,380.0,340.0,NaN,340.0
3,Albania,3626,110.0,115.0,NaN,115.0
4,Algeria,40857,120.0,121.0,172.0,172.0


In [16]:

# France has 327.0 physicians per 10K whilst NY has 389.4?
mask_series = countries_df.country_name.isin(['France'])
countries_df[mask_series]

,country_name,total_population,physicians_per_10k_2009,physicians_per_10k_2013,physicians_per_10k_latest,physicians_per_10k
170,France,227683,370.0,319.0,327.0,327.0


In [9]:

# Remove country duplicates and misspellings
countries_df.country_name = countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
countries_list = sorted(set(countries_df.country_name).symmetric_difference(set(all_countries_df.country_name)))
doubles_df = ssu.check_4_doubles(countries_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False)

,first_item,second_item,max_similarity
12,British Virgin Islands,US Virgin Islands,0.769231
31,Greenland,Grenada,0.750000
16,Cayman Islands,Åland Islands,0.740741
59,Sint Maarten,St. Martin,0.727273
35,Guernsey,Jersey,0.714286
10,Bouvet Island,Faroe Islands,0.692308
25,Faroe Islands,Åland Islands,0.692308
27,French Guiana,French Polynesia,0.620690



## Create Equivalence Dictionaries

In [10]:

state_to_country_equivalent_dict, country_to_state_equivalent_dict = ssu.get_country_state_equivalents(
    countries_df, 'country_name', countries_target_column_name,
    us_states_df, 'state_name', states_target_column_name,
    cn_col_explanation=None, st_col_explanation=None,
    countries_set=None, states_set=None, verbose=False)

In [11]:

string_column_name = 'Country_Equivalent_Physicians_per_Capita'
us_stats_df[string_column_name] = us_stats_df.index.map(lambda x: state_to_country_equivalent_dict.get(x, x))
s.store_objects(us_stats_df=us_stats_df)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\us_stats_df.pkl


In [12]:

states_dict = us_states_df.set_index('state_name')[states_target_column_name].to_dict()
states_min = us_states_df[states_target_column_name].min()
us_stats_df[states_target_column_name] = us_stats_df.index.map(lambda x: states_dict.get(x, states_min))
column_description_dict[states_target_column_name] = 'Physicians per Capita by State (latest)'
s.store_objects(column_description_dict=column_description_dict)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\column_description_dict.pkl



## Choropleth

In [13]:

c.create_label_line_file()
svg_file_path = c.create_country_colored_labeled_map(numeric_column_name=states_target_column_name,
                                                     string_column_name=string_column_name,
                                                     one_country_df=us_stats_df)
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\US_physicians_per_10k_Country_Equivalent_Physicians_per_Capita.svg



----
# Get State/Country Health Care Costs Equivalents


## Clean and prepare US States dataset

In [4]:

url = 'https://worldpopulationreview.com/state-rankings/health-care-costs-by-state'
tables_list = ssu.get_page_tables(url)

[(0, (50, 2))]


In [5]:

us_states_df = tables_list[0].copy()
states_target_column_name = 'spending_per_capita'
us_states_df.columns = ['state_name', states_target_column_name]
us_states_df[states_target_column_name] = pd.to_numeric(us_states_df[states_target_column_name].map(lambda x: re.sub(r'[^0-9\.]+', '', str(x))),
                                                        errors='coerce', downcast='integer')
us_states_df.sample(5)

,state_name,spending_per_capita
30,Indiana,7651.0
40,New Hampshire,7214.0
34,Arizona,7549.0
49,Wyoming,NaN
4,New York,9851.0


In [25]:

# Remove US states duplicates and misspellings
states_list = sorted(set(us_states_df.state_name).symmetric_difference(set(us_stats_df.index)))
doubles_df = ssu.check_4_doubles(states_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False)

,first_item,second_item,max_similarity



## Clean and prepare Countries dataset

In [6]:

driver = ssu.get_driver()
tables_list = ssu.get_page_tables('https://data.worldbank.org/indicator/SH.XPD.CHEX.PC.CD', driver=driver)
driver.close()
if not tables_list:
    tables_list = ssu.get_page_tables('../data/html/world_bank_healthcare_apending_per_capita_by_country.html')

Getting the FireFox driver
No tables found
[]
[(0, (248, 3))]


In [7]:

countries_df = tables_list[0].copy()
countries_target_column_name = 'spending_per_capita'
countries_df.columns = ['country_name', 'study_year', countries_target_column_name]
# print(countries_df.columns.tolist())
countries_df[countries_target_column_name] = pd.to_numeric(countries_df[countries_target_column_name].map(lambda x: re.sub(r'[^0-9\.]+', '',
                                                                                                                           str(x))),
                                                        errors='coerce', downcast='float')
countries_df.sample(5)

,country_name,study_year,spending_per_capita
128,"Micronesia, Fed. Sts.",2019.0,4152.0
138,Nepal,2019.0,5325.0
137,Nauru,2019.0,104945.0
185,Sudan,2019.0,4693.0
1,Albania,2018.0,27491.0


In [ ]:

# Remove country duplicates and misspellings
countries_df.country_name = countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
countries_list = sorted(set(countries_df.country_name).symmetric_difference(set(all_countries_df.country_name)))
doubles_df = ssu.check_4_doubles(countries_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False)


## Create Equivalence Dictionaries

In [10]:

state_to_country_equivalent_dict, country_to_state_equivalent_dict = ssu.get_country_state_equivalents(
    countries_df, 'country_name', countries_target_column_name,
    us_states_df, 'state_name', states_target_column_name,
    cn_col_explanation=None, st_col_explanation=None,
    countries_set=None, states_set=None, verbose=False)

In [11]:

string_column_name = 'Country_Equivalent_Health_Care_Costs'
us_stats_df[string_column_name] = us_stats_df.index.map(lambda x: state_to_country_equivalent_dict.get(x, x))

In [12]:

states_dict = us_states_df.set_index('state_name')[states_target_column_name].to_dict()
states_min = us_states_df[states_target_column_name].min()
us_stats_df[states_target_column_name] = us_stats_df.index.map(lambda x: states_dict.get(x, states_min))
column_description_dict[states_target_column_name] = 'Health Care Costs by State (2022)'
s.store_objects(column_description_dict=column_description_dict)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\column_description_dict.pkl



---
## Choropleth

In [13]:

c.create_label_line_file()
svg_file_path = c.create_country_colored_labeled_map(numeric_column_name=states_target_column_name,
                                                     string_column_name=string_column_name,
                                                     one_country_df=us_stats_df)
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\US_spending_per_capita_Country_Equivalent_Health_Care_Costs.svg



----
# Get State/Country Life Expectency Equivalents


## Clean and prepare US States dataset

In [5]:

url = 'https://worldpopulationreview.com/state-rankings/life-expectancy-by-state'
tables_list = ssu.get_page_tables(url)

[(0, (50, 7))]


In [6]:

life_expectancy_us_states_df = tables_list[0].copy()
life_expectancy_us_states_df.columns = ['state_name', 'life_expectancy', 'life_expectancy_black', 'life_expectancy_latino',
                                        'life_expectancy_asian', 'life_expectancy_native_american', 'life_expectancy_white']
life_expectancy_us_states_df.sample(5)

,state_name,life_expectancy,life_expectancy_black,life_expectancy_latino,life_expectancy_asian,life_expectancy_native_american,life_expectancy_white
33,North Carolina,77.80,74.70,NaN,88.90,76.60,78.30
41,South Carolina,76.20,74.00,NaN,NaN,NaN,77.80
8,Massachusetts,79.90,78.80,87.10,89.10,NaN,80.40
47,Alabama,74.90,72.90,NaN,76.00,NaN,76.00
5,New Jersey,80.40,75.50,84.70,89.40,NaN,80.30



## Clean and prepare Countries dataset

In [7]:

url = 'https://worldpopulationreview.com/countries/life-expectancy'
driver = ssu.get_driver()
tables_list = ssu.get_page_tables(url, driver=driver)
driver.close()

Getting the FireFox driver
[(0, (237, 4))]


In [8]:

life_expectancy_countries_df = tables_list[0].copy()
life_expectancy_countries_df.columns = ['country_name', 'life_expectancy', 'life_expectancy_males', 'life_expectancy_females']
life_expectancy_countries_df.sample(5)

,country_name,life_expectancy,life_expectancy_males,life_expectancy_females
177,Puerto Rico,79.72,75.58,83.90
55,Senegal,67.91,65.47,70.20
13,Seychelles,71.74,68.24,76.04
162,Bahamas,74.36,70.76,77.84
114,Moldova,68.62,64.22,73.32


In [9]:

mask_series = life_expectancy_countries_df.duplicated(subset=['country_name'], keep=False)
life_expectancy_countries_df[mask_series]

,country_name,life_expectancy,life_expectancy_males,life_expectancy_females


In [10]:

life_expectancy_countries_df.country_name = life_expectancy_countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
countries_list = sorted(set(life_expectancy_countries_df.country_name).symmetric_difference(set(all_countries_df.country_name)))
doubles_df = ssu.check_4_doubles(countries_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False)

,first_item,second_item,max_similarity
13,Falkland Islands,Åland Islands,0.83


In [11]:

states_list = sorted(set(life_expectancy_us_states_df.state_name).symmetric_difference(set(us_stats_df.index)))
doubles_df = ssu.check_4_doubles(states_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False)

,first_item,second_item,max_similarity


In [12]:

state_to_country_equivalent_dict, country_to_state_equivalent_dict = ssu.get_country_state_equivalents(life_expectancy_countries_df,
                                                                                                       'country_name',
                                                                                                       'life_expectancy',
                                                                                                       life_expectancy_us_states_df,
                                                                                                       'state_name',
                                                                                                       'life_expectancy',
                                                                                                       verbose=False)
state_to_country_equivalent_dict, country_to_state_equivalent_dict = ssu.get_country_state_equivalents(
    countries_df, 'country_name', countries_target_column_name,
    us_states_df, 'state_name', states_target_column_name,
    cn_col_explanation=None, st_col_explanation=None,
    countries_set=None, states_set=None, verbose=False)

In [13]:

us_stats_df['Country_Equivalent_Life_Expectancy'] = us_stats_df.index.map(lambda x: state_to_country_equivalent_dict.get(x, x))

In [14]:

life_expectancy_dict = life_expectancy_us_states_df.set_index('state_name').life_expectancy.to_dict()
min_life = life_expectancy_us_states_df.life_expectancy.min()
us_stats_df['life_expectancy'] = us_stats_df.index.map(lambda x: life_expectancy_dict.get(x, min_life))
column_description_dict['life_expectancy'] = 'Overall average life expectency (2020)'
s.store_objects(column_description_dict=column_description_dict)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\column_description_dict.pkl



---
## Choropleth

In [15]:

c.create_label_line_file()
numeric_column_name = 'life_expectancy'
string_column_name = 'Country_Equivalent_Life_Expectancy'
svg_file_path = c.create_country_colored_labeled_map(numeric_column_name=numeric_column_name,
                                                     string_column_name=string_column_name,
                                                     one_country_df=us_stats_df)
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\US_life_expectancy_Country_Equivalent_Life_Expectancy.svg



----
# Get State/Country Obesity Equivalents


## Clean and prepare US States dataset

In [51]:

url = 'https://en.wikipedia.org/wiki/Obesity_in_the_United_States'
tables_list = ssu.get_page_tables(url)

[(0, (56, 6)), (1, (24, 2)), (10, (11, 2)), (12, (11, 2)), (4, (9, 2)), (5, (8, 2)), (6, (6, 2)), (8, (4, 2)), (2, (3, 2)), (9, (3, 2)), (11, (2, 2)), (3, (1, 2)), (7, (1, 2))]


In [56]:

us_states_df = tables_list[0].copy()
us_states_df.columns = ['state_name', 'obesity_rank', 'adults_obesity_rate_2005', 'adults_obesity_rate_2020',
                                'adults_overweight_rate_2005', 'children_and_adolescents_obesity_rate_2005']
def f(x):
    rate_float = np.nan
    rate_str = str(x)
    if '%' in rate_str:
        rate_float = float(rate_str.split('%')[0])
    
    return rate_float
for cn in ['adults_obesity_rate_2005', 'adults_obesity_rate_2020',
           'adults_overweight_rate_2005', 'children_and_adolescents_obesity_rate_2005']:
    us_states_df[cn] = us_states_df[cn].map(f)
us_states_df.sample(5)

,state_name,obesity_rank,adults_obesity_rate_2005,adults_obesity_rate_2020,adults_overweight_rate_2005,children_and_adolescents_obesity_rate_2005
19,Kentucky,8,28.4,34.3,66.8,20.6
0,Alabama,5,30.1,36.3,65.4,16.7
14,Idaho,32,24.6,29.3,61.4,10.1
35,North Carolina,20,27.1,32.1,63.4,19.3
45,South Dakota,22,26.1,31.9,64.2,12.1


In [57]:

mask_series = (us_states_df.state_name == 'District of Columbia')
us_states_df = us_states_df[~mask_series]

In [58]:

# Remove US states duplicates and misspellings
states_list = sorted(set(us_states_df.state_name).symmetric_difference(set(ssu.us_stats_df.index)))
print(states_list)
doubles_df = ssu.check_4_doubles(states_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = us_states_df.duplicated(subset=['state_name'], keep=False)
if us_states_df[mask_series].shape[0]:
    display(us_states_df[mask_series])

['American Samoa', 'District of Columbia', 'Guam', 'Northern Mariana Islands', 'Puerto Rico', 'Virgin Islands (U.S.)']


In [60]:

s.store_objects(obesity_us_states_df=us_states_df)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\obesity_us_states_df.pkl



## Clean and prepare Countries dataset

In [61]:

url = 'https://en.wikipedia.org/wiki/List_of_countries_by_obesity_rate'
# driver = ssu.get_driver()
tables_list = ssu.get_page_tables(url, driver=None)
# driver.close()

[(0, (191, 3)), (1, (12, 2))]


In [62]:

countries_df = tables_list[0].copy()
countries_df.columns = ['country_name', 'obesity_rank', 'obesity_rate_2016']
countries_df.obesity_rate_2016 = countries_df.obesity_rate_2016.map(lambda x: float(x))
countries_df.sample(5)

,country_name,obesity_rank,obesity_rate_2016
8,Kiribati,9,46.0
25,Canada,26,29.4
1,Cook Islands,2,55.9
136,Mauritius,137,10.8
155,Equatorial Guinea,156,8.0


In [63]:

# Remove country duplicates and misspellings
countries_df.country_name = countries_df.country_name.map(lambda x: ssu.country_name_dict.get(x, x))
countries_list = sorted(set(countries_df.country_name).symmetric_difference(set(all_countries_df.country_name)))
doubles_df = ssu.check_4_doubles(countries_list)
mask_series = (doubles_df.max_similarity > 0.6)
columns_list = ['first_item', 'second_item', 'max_similarity']
if doubles_df[mask_series].shape[0]:
    display(doubles_df[mask_series][columns_list].sort_values('max_similarity', ascending=False))
mask_series = countries_df.duplicated(subset=['country_name'], keep=False)
if countries_df[mask_series].shape[0]:
    display(countries_df[mask_series])

,first_item,second_item,max_similarity
8,British Virgin Islands,US Virgin Islands,0.769231
10,Cayman Islands,Åland Islands,0.740741
48,Sint Maarten,St. Martin,0.727273
24,Guernsey,Jersey,0.714286
47,San Marino,St. Martin,0.700000
6,Bouvet Island,Faroe Islands,0.692308
16,Faroe Islands,Åland Islands,0.692308
17,French Guiana,French Polynesia,0.620690


In [64]:

s.store_objects(obesity_countries_df=countries_df)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\pkl\obesity_countries_df.pkl



## Prepare for and Create Choropleth

In [34]:

state_to_country_equivalent_dict, country_to_state_equivalent_dict = ssu.get_country_state_equivalents(countries_df, 'country_name',
                                                                                                       'obesity_rate_2016',
                                                                                                       us_states_df, 'state_name',
                                                                                                       'adults_obesity_rate_2005',
                                                                                                       verbose=False)
state_to_country_equivalent_dict, country_to_state_equivalent_dict = ssu.get_country_state_equivalents(
    countries_df, 'country_name', countries_target_column_name,
    us_states_df, 'state_name', states_target_column_name,
    cn_col_explanation=None, st_col_explanation=None,
    countries_set=None, states_set=None, verbose=False)

In [38]:

column_description_dict['adults_obesity_rate_2020'] = 'Obese adults (2020)'
column_description_dict['adults_obesity_rate_2005'] = 'Obese adults (mid-2000s)'
s.store_objects(column_description_dict=column_description_dict)
us_stats_df['Country_Equivalent_Obesity'] = us_stats_df.index.map(lambda x: state_to_country_equivalent_dict.get(x, x))

In [39]:

obesity_dict = us_states_df.set_index('state_name').adults_obesity_rate_2020.to_dict()
obesity_min = us_states_df.adults_obesity_rate_2020.min()
us_stats_df['adults_obesity_rate_2020'] = us_stats_df.index.map(lambda x: obesity_dict.get(x, obesity_min))

In [40]:

obesity_dict = us_states_df.set_index('state_name').adults_obesity_rate_2005.to_dict()
obesity_min = us_states_df.adults_obesity_rate_2005.min()
us_stats_df['adults_obesity_rate_2005'] = us_stats_df.index.map(lambda x: obesity_dict.get(x, obesity_min))


---
## Choropleth

In [41]:

c.create_label_line_file()
numeric_column_name = 'adults_obesity_rate_2005'
string_column_name = 'Country_Equivalent_Obesity'
svg_file_path = c.create_country_colored_labeled_map(numeric_column_name=numeric_column_name,
                                                     string_column_name=string_column_name,
                                                     one_country_df=us_stats_df)
print(os.path.abspath(svg_file_path))

C:\Users\daveb\OneDrive\Documents\GitHub\StatsByCountry\saves\svg\US_adults_obesity_rate_2005_Country_Equivalent_Obesity.svg



----
# Get State/Country Student Loan Debt Equivalents

In [17]:

url = 'https://educationdata.org/student-loan-debt-by-state'
tables_list = ssu.get_page_tables(url)

[(0, (53, 3))]


In [18]:

student_load_debt_us_states_df = tables_list[0].copy()
student_load_debt_us_states_df.sample(5)

,State,Average Borrower Debt,State’s Total Debt
10,Vermont,"$37,516",$2.9 billion
29,New Hampshire,"$34,085",$6.5 billion
52,Other/Unspecified*,"$25,960",$99.0 billion
36,Idaho,"$33,012",$7.2 billion
2,Georgia,"$41,639",$68.6 billion



----

In [10]:

us_stats_df.index

Index(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia',
       'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky',
       'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
       'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype='object', name='state_name')


---
# Choropleths

In [13]:

column_descriptions_df = ssu.get_column_descriptions(c.one_country_df)
mask_series = (column_descriptions_df.dtype == 'object')
print(column_descriptions_df[mask_series].column_name.tolist())
print()
print(column_descriptions_df[~mask_series].column_name.tolist())

['district_abbreviation', 'State_Region', 'Google_Suggest_Unique', 'label_line_d', 'Google_Suggest_Common', 'Google_Suggest_First', 'outline_d', 'Country_Equivalent_GDP', 'Country_Equivalent_Military_Expenditure', 'state_color', 'centroid_id']

['White_Percent', 'Black_Percent', 'Hispanic_Percent', 'Asian_Percent', 'Native_Percent', 'Islander_Percent', 'Multi_Percent', 'Gini_Index', 'Effectiveness_Rank', 'Health_Care_Score', 'Education_Score', 'Economy_Score', 'Infrastructure_Score', 'Opportunity_Score', 'Fiscal_Stability_Score', 'Crime_Corrections_Score', 'Natural_Environment_Score', 'GDP_Rank', 'GDP_2018', 'GDP_Percent', 'Homicide_Rate_2018', 'Homicide_Rate_2017', 'Homicide_Rate_2014', 'Homicide_Rate_2010', 'Homicide_Rate_2005', 'Homicide_Rate_2000', 'Homicide_Rate_1996', 'Guns_Rank', 'Guns_Per_Capita', 'Guns_Registered', 'Suicide_Rate_2017', 'Suicide_Deaths_2017', 'Suicide_Rate_2016', 'Suicide_Deaths_2016', 'Suicide_Rate_2015', 'Suicide_Deaths_2015', 'Suicide_Rate_2014', 'Suicide_De

In [10]:

for file_name in os.listdir(s.saves_pickle_folder):
    if file_name.endswith('_df.pkl'):
        df_name = file_name.split('.')[0]
        df = s.load_object(df_name)
        columns_list = df.columns.tolist()
        if 'district_abbreviation' in columns_list:
            print(f'{df_name}: {columns_list}')

us_stats_df: ['White_Percent', 'Black_Percent', 'Hispanic_Percent', 'Asian_Percent', 'Native_Percent', 'Islander_Percent', 'Multi_Percent', 'Gini_Index', 'Effectiveness_Rank', 'Health_Care_Score', 'Education_Score', 'Economy_Score', 'Infrastructure_Score', 'Opportunity_Score', 'Fiscal_Stability_Score', 'Crime_Corrections_Score', 'Natural_Environment_Score', 'district_abbreviation', 'GDP_Rank', 'GDP_2018', 'GDP_Percent', 'State_Region', 'Homicide_Rate_2018', 'Homicide_Rate_2017', 'Homicide_Rate_2014', 'Homicide_Rate_2010', 'Homicide_Rate_2005', 'Homicide_Rate_2000', 'Homicide_Rate_1996', 'Guns_Rank', 'Guns_Per_Capita', 'Guns_Registered', 'Suicide_Rate_2017', 'Suicide_Deaths_2017', 'Suicide_Rate_2016', 'Suicide_Deaths_2016', 'Suicide_Rate_2015', 'Suicide_Deaths_2015', 'Suicide_Rate_2014', 'Suicide_Deaths_2014', 'Suicide_Rate_2005', 'Suicide_Deaths_2005', 'Total_Inhabitants_2010', 'Inhabitants_Per_Square_Mile_2010', 'Total_Murder_Deaths_2010', 'Total_Gun_Murder_Deaths_2010', 'Gun_Ownershi

In [11]:

text_editor_path = r'C:\Program Files\Notepad++\notepad++.exe'
inkscape_path = r'C:\Program Files\Inkscape\bin\inkscape.exe'

In [ ]:

column_name = 'Estimated_IQ'
if column_name in us_stats_df.columns:
    svg_file_path = c.create_country_colored_map(column_name)
    !"{text_editor_path}" "{os.path.abspath(svg_file_path)}"

In [ ]:

column_name = 'White_Percent'
if column_name in us_stats_df.columns:
    svg_file_path = os.path.abspath(c.create_country_colored_map(column_name))
    !"{text_editor_path}" "{svg_file_path}"
    # !"{inkscape_path}" window-open "{svg_file_path}"

In [ ]:

column_name = 'Percent_Whites_in_Non_Public_Education'
if column_name in us_stats_df.columns:
    svg_file_path = os.path.abspath(c.create_country_colored_map(column_name))
    !"{text_editor_path}" "{svg_file_path}"
    # !"{inkscape_path}" window-open "{svg_file_path}"

In [ ]:

if 'centroid_id' not in us_stats_df.columns:
    us_stats_df['centroid_id'] = us_stats_df.index.map(lambda x: ('district-' + c.indexize_string(x)).replace('-district', ''))
    s.store_objects(us_stats_df=us_stats_df)

In [ ]:

if 'dy' not in us_stats_df.columns:
    us_stats_df['dy'] = np.nan
    s.store_objects(us_stats_df=us_stats_df)

In [ ]:

string_column_name = 'State_Region'
if string_column_name in us_stats_df.columns:
    c.create_label_line_file()
    svg_file_path = os.path.abspath(c.create_country_labeled_map(string_column_name=string_column_name,
                                                                 one_country_df=c.one_country_df))
    !"{text_editor_path}" "{svg_file_path}"
    !"{inkscape_path}" window-open "{svg_file_path}"

In [ ]:

numeric_column_name = 'Asian_Percent'
string_column_name = 'Google_Suggest_Common'
if (numeric_column_name in us_stats_df.columns) and (string_column_name in us_stats_df.columns):
    svg_file_path = os.path.abspath(c.create_country_colored_labeled_map(numeric_column_name=numeric_column_name,
                                                                         string_column_name=string_column_name,
                                                                         one_country_df=c.one_country_df))
    !"{text_editor_path}" "{svg_file_path}"

In [14]:

state_color_dict = s.load_object('us_state_name_color_dict')
string_column_name = 'Google_Suggest_First'
if string_column_name in us_stats_df.columns:
    svg_file_path = os.path.abspath(c.create_country_labeled_map(string_column_name=string_column_name,
                                                                 one_country_df=c.one_country_df))
    !"{text_editor_path}" "{svg_file_path}"

In [ ]:

from matplotlib import cm

ListedColormap_obj = cm.get_cmap('viridis', len(c.one_country_df.State_Budget_Processes.unique()))
min = c.one_country_df.State_Budget_Processes.min()
max = c.one_country_df.State_Budget_Processes.max()
normed_series = (c.one_country_df.State_Budget_Processes - min) / (max - min)
sample_value = normed_series.sample(1).tolist()[0]
if str(sample_value) != 'nan':
    print(ListedColormap_obj(sample_value), '#{:02x}{:02x}{:02x}{:02x}'.format(*tuple(int(x*255) for x in ListedColormap_obj(sample_value))))

In [ ]:

print(['c.one_country_df.{}'.format(fn) for fn in dir(c.one_country_df) if 'sort' in fn.lower()])

In [ ]:

!start %windir%\explorer.exe "{c.svg_dir}"

In [ ]:

for column_name in c.one_country_df.columns:
    svg_file_path = c.create_country_colored_map(column_name=column_name)

In [ ]:

print(['c.one_country_df.{}'.format(cn) for cn in c.one_country_df.columns if ('gun' in cn.lower()) and ('murder' in cn.lower())])


---
# O Canada!

In [9]:

page_tables_list = ssu.get_page_tables('https://en.wikipedia.org/wiki/Demographics_of_Canada')

[(22, (140, 8)), (14, (122, 9)), (26, (78, 2)), (27, (37, 2)), (4, (28, 11)), (0, (26, 2)), (5, (25, 12)), (28, (25, 2)), (17, (24, 3)), (13, (22, 5)), (35, (21, 2)), (12, (18, 11)), (25, (18, 3)), (18, (17, 11)), (1, (14, 11)), (41, (13, 2)), (16, (11, 3)), (2, (10, 10)), (3, (10, 10)), (6, (8, 3)), (20, (8, 5)), (10, (7, 3)), (7, (6, 3)), (8, (6, 3)), (23, (6, 3)), (24, (6, 3)), (32, (6, 2)), (11, (5, 3)), (21, (5, 5)), (29, (5, 2)), (31, (5, 2)), (9, (4, 3)), (19, (4, 11)), (34, (4, 2)), (15, (3, 4)), (30, (3, 2)), (33, (3, 2)), (36, (2, 2)), (37, (2, 2)), (38, (2, 2)), (39, (2, 2)), (40, (2, 2)), (42, (2, 2))]


In [ ]:

canada_races_df = page_tables_list[14].copy()
canada_races_df.columns = [cn.split('[')[0].strip() for cn in canada_races_df.columns.droplevel(0).tolist()]
canada_races_df['Province/territory'] = canada_races_df['Province/territory'].map(lambda cn: cn.split('[')[0])
canada_races_df.set_index('Province/territory', drop=True, inplace=True)
canada_races_df['Percent visible minority'] = canada_races_df['Percent visible minority'].map(lambda x: float(str(x).split('%')[0]))
canada_races_df['Percent_White'] = canada_races_df['Percent visible minority'].map(lambda x: 100.0 - x)
canada_races_df.dropna(axis='columns', how='all', inplace=True)
canada_races_df.sample(5).T

In [ ]:

s.store_objects(canada_races_df=canada_races_df)


---
# Add New Columns

In [ ]:

us_stats_df.columns = ['White_Percent', 'Black_Percent', 'Hispanic_Percent', 'Asian_Percent', 'Native_Percent', 'Islander_Percent',
                       'Multi_Percent', 'Gini_Index', 'Effectiveness_Rank', 'Health_Care_Score', 'Education_Score',
                       'Economy_Score', 'Infrastructure_Score', 'Opportunity_Score', 'Fiscal_Stability_Score',
                       'Crime_Corrections_Score', 'Natural_Environment_Score', 'district_abbreviation', 'GDP_Rank', 'GDP_2018',
                       'GDP_Percent', 'State_Region', 'Homicide_Rate_2018', 'Homicide_Rate_2017', 'Homicide_Rate_2014',
                       'Homicide_Rate_2010', 'Homicide_Rate_2005', 'Homicide_Rate_2000', 'Homicide_Rate_1996', 'Guns_Rank',
                       'Guns_Per_Capita', 'Guns_Registered', 'Suicide_Rate_2017', 'Suicide_Deaths_2017', 'Suicide_Rate_2016',
                       'Suicide_Deaths_2016', 'Suicide_Rate_2015', 'Suicide_Deaths_2015', 'Suicide_Rate_2014',
                       'Suicide_Deaths_2014', 'Suicide_Rate_2005', 'Suicide_Deaths_2005',
                       'Total_Inhabitants_2010', 'Inhabitants_Per_Square_Mile_2010', 'Total_Murder_Deaths_2010',
                       'Total_Gun_Murder_Deaths_2010', 'Gun_Ownership_Percent_2013', 'Murder_Rate_2010', 'Gun_Murder_Rate_2010',
                       'State_FIPS', 'State_Population', 'Gun_Suicide_Deaths', 'Gun_Suicide_Rate', 'Google_Suggest_Unique', 'text_x',
                       'text_y', 'label_line_d', 'Google_Suggest_Common', 'Google_Suggest_First',
                       'Public_Access_to_Information', 'Political_Financing', 'Electoral_Oversight',
                       'Executive_Accountability', 'Legislative_Accountability', 'Judicial_Accountability',
                       'State_Budget_Processes', 'State_Civil_Service_Management', 'Procurement', 'Internal_Auditing',
                       'Lobbying_Disclosure', 'Ethics_Enforcement_Entities', 'State_Pension_Fund_Management', 'outline_d',
                       'centroid_x', 'centroid_y']
s.store_objects(us_stats_df=us_stats_df)

In [ ]:

import xml.etree.ElementTree as et

file_path = os.path.join(s.data_folder, 'svg', 'us.svg')
#print(['root.{}'.format(fn) for fn in dir(root) if not fn.startswith('_')])
us_stats_df = s.load_object('us_stats_df')
root = et.parse(file_path).getroot()
outline_d_dict = {}
for tag in root:
    if (tag.tag.split('}')[-1] == 'path'):
        #print(['tag.{}'.format(fn) for fn in dir(tag) if not fn.startswith('_')])
        state_name = tag.attrib['data-name']
        outline_d = tag.attrib['d']
        outline_d_dict[state_name] = outline_d

df = pd.DataFrame([outline_d_dict]).T
df.columns = ['outline_d']
us_stats_df = us_stats_df.T.append(df.T).T
us_stats_df.T.tail(5).T.sample(7).T

In [ ]:

s.store_objects(us_stats_df=us_stats_df)


---

In [ ]:

file_name = 'State_Integrity_2015_Full_Dataset.xlsx'
excel_path = os.path.join(s.data_folder, 'xlsx', file_name)
sheet_df_dict = pd.read_excel(excel_path, sheet_name=None)

In [ ]:

us_stats_df = s.load_object('us_stats_df')
for sheet_name in sheet_df_dict.keys():
    column_name = '_'.join(sheet_name.strip().split(' '))
    df = sheet_df_dict[sheet_name].copy()
    index_columns = df.loc[0, ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2']].tolist()
    df.columns = index_columns + df.columns.tolist()[3:]
    df.set_index(keys=index_columns, inplace=True)
    cn_dict = {}
    for state_name in us_stats_df.index:
        cn_dict[state_name] = []
    for index_tuple, row_series in df[states_list].iterrows():
        if str(index_tuple[1]).isdigit():
            #print(index_tuple)
            for state_name, column_value in row_series.iteritems():
                state_name = state_name.strip()
                column_value = column_value.strip()
                if str(column_value).isdigit():
                    scores_list = cn_dict[state_name]
                    scores_list.append(column_value)
                    cn_dict[state_name] = scores_list
                elif column_value.lower() in ['no', 'moderate', 'yes']:
                    scores_list = cn_dict[state_name]
                    scores_list.append(['no', 'moderate', 'yes'].index(column_value.lower())*50)
                    cn_dict[state_name] = scores_list
    cn_dict = {state_name: sum([b/len(scores_list) for b in scores_list]) for state_name,
               scores_list in cn_dict.items()}
    df = pd.DataFrame([cn_dict]).T
    df.columns = [column_name]
    us_stats_df = us_stats_df.T.append(df.T).T

In [ ]:

us_stats_df.T.tail(20).T.sample(8).T


---
# Get Correlations ("P-Hunting")

In [ ]:

def get_correlation_dataframe(numeric_columns_list):
    rows_list = []
    for x_column in numeric_columns_list:
        for y_column in numeric_columns_list:
            if x_column != y_column:
                columns_list = [x_column, y_column]
                df = us_stats_df[columns_list].dropna()
                x = df[x_column].values
                y = df[y_column].values
                try:
                    r_tuple = stats.pearsonr(x, y)
                    if r_tuple[1] < 0.05:
                        row_dict = {}
                        row_dict['left_column'] = x_column
                        row_dict['right_column'] = y_column
                        row_dict['pearson_r'] = abs(r_tuple[0])
                        rows_list.append(row_dict)
                except Exception as e:
                    print('{} and {} get an error: {}'.format(x_column, y_column, e))
    correlation_df = pd.DataFrame(rows_list, columns=['left_column', 'right_column', 'pearson_r'])
    
    return correlation_df

In [ ]:

column_descriptions_df = ssu.get_column_descriptions(df=us_stats_df)
column_descriptions_df.columns

In [ ]:

mask_series = (column_descriptions_df.dtype == 'float32')
columns_list = [row_series.column_name for row_index, row_series in column_descriptions_df[mask_series].iterrows()]
correlation_df = get_correlation_dataframe(columns_list)

In [ ]:

mask_series = (correlation_df.pearson_r > 0.95)
correlation_df[mask_series].sort_values('pearson_r', ascending=False).left_column.tolist()

In [ ]:

def get_correlation_dataframe(x_column_list, y_column_list):
    rows_list = []
    for x_column in x_column_list:
        for y_column in y_column_list:
            if x_column != y_column:
                columns_list = [x_column, y_column]
                df = us_stats_df[columns_list].dropna()
                x = df[x_column].values
                y = df[y_column].values
                try:
                    r_tuple = stats.pearsonr(x, y)
                    if r_tuple[1] < 0.05:
                        row_dict = {}
                        row_dict['left_column'] = x_column
                        row_dict['right_column'] = y_column
                        row_dict['pearson_r'] = abs(r_tuple[0])
                        rows_list.append(row_dict)
                except Exception as e:
                    print('{} and {} get an error: {}'.format(x_column, y_column, e))
    correlation_df = pd.DataFrame(rows_list, columns=['left_column', 'right_column', 'pearson_r'])
    
    return correlation_df

In [ ]:

mask_series = (column_descriptions_df.dtype == 'float32')
numeric_columns_list = [row_series.column_name for row_index, row_series in column_descriptions_df[mask_series].iterrows()]
new_columns_list = us_stats_df.T.tail(13).index.tolist()
old_columns_list = list(set(numeric_columns_list) - set(new_columns_list))
correlation_df = get_correlation_dataframe(new_columns_list, old_columns_list)
correlation_df.sort_values('pearson_r', ascending=False)


---
# Linear Scatterplots

In [ ]:

def show_linear_scatterplot(merged_df, columns_list, ev_min_str=None, ev_max_str=None, rv_min_str=None, rv_max_str=None):
    ev_column_name = columns_list[0]
    rv_column_name = columns_list[1]
    explanatory_variable = get_column_description(ev_column_name)
    response_variable = get_column_description(rv_column_name)
    if (ev_min_str is None):
        ev_min_str = 'minimum {}'.format(explanatory_variable)
    if (ev_max_str is None):
        ev_max_str = 'maximum {}'.format(explanatory_variable)
    if (rv_min_str is None):
        rv_min_str = 'minimum {}'.format(response_variable)
    if (rv_max_str is None):
        rv_max_str = 'maximum {}'.format(response_variable)
    
    df = merged_df.copy()
    columns_list = [ev_column_name, rv_column_name]
    df = df[columns_list].dropna()
    ev_max = df[ev_column_name].max()
    ev_min = df[ev_column_name].min()
    rv_min = df[rv_column_name].min()
    rv_max = df[rv_column_name].max()
    ev_max_labeled = False
    ev_min_labeled = False
    rv_min_labeled = False
    rv_max_labeled = False
    
    # First order (linear) scatterplot
    fig1_fig = plt.figure(figsize=(12, 8))
    merge_axes_subplot = sns.regplot(x=ev_column_name, y=rv_column_name,
                                     scatter=True, data=df)
    xlabel_text = plt.xlabel('{} (explanatory variable)'.format(explanatory_variable))
    ylabel_text = plt.ylabel('{} (response variable)'.format(response_variable))
    
    # Add annotations
    for label, x, y in zip(df.index, df[ev_column_name], df[rv_column_name]):
        if (x == ev_min):
            if not ev_min_labeled:
                ev_min_labeled = True
                annotation = plt.annotate('{} ({})'.format(label, ev_min_str), xy=(x, y), xytext=ev_min_xytext, **kwargs)
        elif (x == ev_max):
            if not ev_max_labeled:
                ev_max_labeled = True
                annotation = plt.annotate('{} ({})'.format(label, ev_max_str), xy=(x, y), xytext=ev_max_xytext, **kwargs)
        elif (y == rv_min):
            if not rv_min_labeled:
                rv_min_labeled = True
                annotation = plt.annotate('{} ({})'.format(label, rv_min_str), xy=(x, y), xytext=rv_min_xytext, **kwargs)
        elif (y == rv_max):
            if not rv_max_labeled:
                rv_max_labeled = True
                annotation = plt.annotate('{} ({})'.format(label, rv_max_str), xy=(x, y), xytext=rv_max_xytext, **kwargs)
        elif (label == 'Arizona'):
            annotation = plt.annotate('{} (my home state)'.format(label), xy=(x, y), xytext=az_xytext, **kwargs)
    
    # Add r-squared
    x = df[ev_column_name].values
    y = df[rv_column_name].values
    plt.text(0.92, 0.965, r'$r^2 = {0:.2}$'.format(stats.pearsonr(x, y)[0] ** 2), fontsize=20, alpha=0.25,
             horizontalalignment='center', verticalalignment='center', transform=merge_axes_subplot.transAxes)

In [ ]:

def get_column_description(column_name):
    if column_name in column_description_dict:
        column_description = column_description_dict[column_name]
    else:
        column_description = re.sub('^pf_', 'Personal Freedom:_', str(column_name), 1)
        column_description = re.sub('^hf_', 'Human Freedom:_', str(column_description), 1)
        column_description = re.sub('^ef_', 'Economic Freedom:_', str(column_description), 1)
        column_list = column_description.split('_')
        descr_list = []
        for word in column_list:
            descr_list.append(word[0].upper()+word[1:])
        column_description = ' '.join(descr_list)
        column_description_dict[column_name] = column_description
        s.store_objects(column_description_dict=column_description_dict)
    
    return column_description

In [ ]:

from scipy import stats

kwargs = dict(textcoords='offset points', ha='left', va='bottom',
              bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
              arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

ev_column_name = 'State_Budget_Processes'

rv_column_name = 'Homicide_Rate_2018'

ev_min_str = 'worst process'
ev_max_str = 'best process'
rv_min_str = 'least murderous'
rv_max_str = 'most murderous'
ev_min_xytext = (-5, -40)
ev_max_xytext = (-100, -40)
rv_min_xytext = (-45, -35)
rv_max_xytext = (-100, -50)
az_xytext = (0, 30)
columns_list = [ev_column_name, rv_column_name]
show_linear_scatterplot(us_stats_df, columns_list, ev_min_str=ev_min_str,
                        ev_max_str=ev_max_str, rv_min_str=rv_min_str, rv_max_str=rv_max_str)

In [ ]:

correlation_df = get_correlation_dataframe(new_columns_list, new_columns_list)
correlation_df.sort_values('pearson_r', ascending=False)

In [ ]:

from scipy import stats

kwargs = dict(textcoords='offset points', ha='left', va='bottom',
              bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
              arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

ev_column_name = 'Executive_Accountability'

rv_column_name = 'Legislative_Accountability'

ev_min_str = 'least executive accountability'
ev_max_str = 'most executive accountability'
rv_min_str = 'least legislative accountability'
rv_max_str = 'most legislative accountability'
ev_min_xytext = (15, 5)
ev_max_xytext = (-210, -30)
rv_min_xytext = (-45, -35)
rv_max_xytext = (-100, -50)
az_xytext = (-15, 10)
columns_list = [ev_column_name, rv_column_name]
show_linear_scatterplot(us_stats_df, columns_list, ev_min_str=ev_min_str,
                        ev_max_str=ev_max_str, rv_min_str=rv_min_str, rv_max_str=rv_max_str)

In [ ]:

kwargs = dict(textcoords='offset points', ha='left', va='bottom',
              bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
              arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

ev_column_name = 'Guns_Registered'
rv_column_name = 'Suicide_Deaths_2017'

ev_min_str = 'least gun-nutty'
ev_max_str = 'most gun-nutty'
rv_min_str = 'least suicidal'
rv_max_str = 'most suicidal'
ev_min_xytext = (50, -10)
ev_max_xytext = (-150, -60)
rv_min_xytext = (20, -15)
rv_max_xytext = (100, -50)
az_xytext = (60, 50)
columns_list = [ev_column_name, rv_column_name]
df = us_stats_df[columns_list+['Total_Inhabitants_2010']].copy()
df[ev_column_name] = df[ev_column_name]/df['Total_Inhabitants_2010']
df[rv_column_name] = df[rv_column_name]/df['Total_Inhabitants_2010']
mask_series = (df.index == 'Wyoming')
show_linear_scatterplot(df[~mask_series], columns_list, ev_min_str=ev_min_str,
                        ev_max_str=ev_max_str, rv_min_str=rv_min_str, rv_max_str=rv_max_str)

In [10]:

us_stats_df = s.load_object('us_stats_df')
tables_url = 'https://en.wikipedia.org/wiki/Firearm_death_rates_in_the_United_States_by_state'
tables_df_list = ssu.get_page_tables(tables_url, verbose=True)

[(1, (52, 8)), (0, (50, 4))]


In [ ]:

us_stats_df.district_abbreviation.to_dict()

In [ ]:

gun_murders_df = tables_df_list[4].copy()
gun_murders_df.set_index('State', inplace=True)
print(gun_murders_df.columns.tolist())
gun_murders_df.columns = ['Total_Inhabitants_2010', 'Inhabitants_Per_Square_Mile_2010', 'Total_Murder_Deaths_2010',
                          'Total_Gun_Murder_Deaths_2010', 'Gun_Ownership_Percent_2013', 'Murder_Rate_2010',
                          'Gun_Murder_Rate_2010']
gun_murders_df.Gun_Ownership_Percent_2013 = gun_murders_df.Gun_Ownership_Percent_2013.map(lambda x: float(str(x).split('%')[0]))
gun_murders_df.sample(5).T

In [ ]:

gun_suicides_df = s.load_csv(csv_name='gun_suicides_by_state',
                             folder_path=s.data_folder).dropna(axis=0, how='all').dropna(axis=1, how='all')
gun_suicides_df = gun_suicides_df.iloc[11:].dropna(axis=1, how='all')
abbrev_dict = {row_series.district_abbreviation: state_name for state_name, row_series in us_stats_df.iterrows()}
abbrev_dict['DC'] = 'District of Columbia'
gun_suicides_df.ST = gun_suicides_df.ST.map(lambda x: abbrev_dict[x])
gun_suicides_df.columns = ['State', 'State_FIPS', 'State_Population', 'Gun_Suicide_Deaths', 'Gun_Suicide_Rate']
gun_suicides_df.set_index('State', inplace=True)
gun_suicides_df.Gun_Suicide_Rate = gun_suicides_df.Gun_Suicide_Rate.map(lambda x: float(x))
for column_name in ['State_FIPS', 'State_Population', 'Gun_Suicide_Deaths']:
    gun_suicides_df[column_name] = gun_suicides_df[column_name].map(lambda x: int(x))
gun_suicides_df.sample(5).T

In [ ]:

set(gun_merge_df.columns).intersection(set(us_stats_df.columns))

In [ ]:

columns_list = ['White_Percent', 'Black_Percent', 'Hispanic_Percent', 'Asian_Percent', 'Native_Percent', 'Islander_Percent',
                'Multi_Percent', 'Gini_Index', 'Effectiveness_Rank', 'Health_Care_Score', 'Education_Score', 'Economy_Score',
                'Infrastructure_Score', 'Opportunity_Score', 'Fiscal_Stability_Score', 'Crime_Corrections_Score',
                'Natural_Environment_Score', 'district_abbreviation', 'GDP_Rank', 'GDP_2018', 'GDP_Percent', 'State_Region',
                'Homicide_Rate_2018', 'Homicide_Rate_2017', 'Homicide_Rate_2014', 'Homicide_Rate_2010', 'Homicide_Rate_2005',
                'Homicide_Rate_2000', 'Homicide_Rate_1996', 'Guns_Rank', 'Guns_Per_Capita', 'Guns_Registered',
                'Suicide_Rate_2017', 'Suicide_Deaths_2017', 'Suicide_Rate_2016', 'Suicide_Deaths_2016', 'Suicide_Rate_2015',
                'Suicide_Deaths_2015', 'Suicide_Rate_2014', 'Suicide_Deaths_2014', 'Suicide_Rate_2005', 'Suicide_Deaths_2005']
us_stats_df = pd.merge(left=us_stats_df[columns_list], right=gun_merge_df, left_index=True,
                          right_index=True, suffixes=('_merge', '_guns'))

In [ ]:

us_stats_df.columns.tolist()

In [ ]:

column_descriptions_df = ssu.get_column_descriptions(df=us_stats_df, column_list=us_stats_df.columns)
mask_series = (column_descriptions_df.dtype.isin(['int64', 'float64']))
print(column_descriptions_df[~mask_series].column_name.tolist())
column_descriptions_df[~mask_series]

In [ ]:

column_descriptions_df = ssu.get_column_descriptions(us_stats_df)
column_descriptions_df['dtype'].unique()

In [ ]:

mask_series = (column_descriptions_df['dtype'].isin(['int64', 'float64']))
numeric_columns_list = column_descriptions_df[mask_series]['column_name'].tolist()

In [ ]:

for column_name in numeric_columns_list:
    us_stats_df[column_name] = pd.to_numeric(us_stats_df[column_name])

In [ ]:

from scipy import stats

r_columns_list = []
rows_list = []
for x_column in numeric_columns_list:
    for y_column in numeric_columns_list:
        if x_column != y_column:
            columns_list = [x_column, y_column]
            df = us_stats_df[columns_list].dropna()
            x = df[x_column].values
            y = df[y_column].values
            try:
                r_tuple = stats.pearsonr(x, y)
                if r_tuple[1] < 0.05:
                    c_tuple = ('/'.join(columns_list), row_dict['pearson_r'])
                    r_columns_list.append(c_tuple)
            except Exception as e:
                print('{} and {} get an error: {}'.format(x_column, y_column, e))

In [ ]:

column_pairs_list = sorted(r_columns_list, key=lambda x: x[1], reverse=True)
[column_pairs_list[0][0].split('/')[0], column_pairs_list[0][0].split('/')[1]]

In [ ]:

us_stats_df = s.load_object('us_stats_df')
kwargs = dict(textcoords='offset points', ha='left', va='bottom',
              bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
              arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

ev_column_name = 'Guns_Registered'
column_description_dict[ev_column_name] = 'Number of Guns Registered'
#s.store_objects(column_description_dict=column_description_dict)

rv_column_name = 'Gun_Murder_Rate_2010'
column_description_dict[rv_column_name] = 'Gun Murder Rate'
s.store_objects(column_description_dict=column_description_dict)

ev_min_str = 'least gun-nutty'
ev_max_str = 'most gun-nutty'
rv_min_str = 'least murderous'
rv_max_str = 'most murderous'
ev_min_xytext = (-5, 90)
ev_max_xytext = (-130, -60)
rv_min_xytext = (20, -15)
rv_max_xytext = (-100, -50)
az_xytext = (-60, 50)
columns_list = [ev_column_name, rv_column_name]
show_linear_scatterplot(us_stats_df, columns_list, ev_min_str=ev_min_str,
                        ev_max_str=ev_max_str, rv_min_str=rv_min_str, rv_max_str=rv_max_str)

In [ ]:

import re
from scipy import stats
%run ../load_magic/dataframes.py

us_stats_df = s.load_object('us_stats_df')
kwargs = dict(textcoords='offset points', ha='left', va='bottom',
              bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
              arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

ev_column_name = 'Guns_Registered'
column_description_dict[ev_column_name] = 'Number of Guns Registered'
#s.store_objects(column_description_dict=column_description_dict)

rv_column_name = 'Gun_Suicide_Deaths'
column_description_dict[rv_column_name] = 'Gun Suicide Deaths'
s.store_objects(column_description_dict=column_description_dict)

ev_min_str = 'least gun-nutty'
ev_max_str = 'most gun-nutty'
rv_min_str = 'least suicidal'
rv_max_str = 'most suicidal'
ev_min_xytext = (-5, 110)
ev_max_xytext = (-130, -60)
rv_min_xytext = (20, -30)
rv_max_xytext = (-100, -50)
az_xytext = (-60, 50)
columns_list = [ev_column_name, rv_column_name]
show_linear_scatterplot(us_stats_df, columns_list, ev_min_str=ev_min_str,
                        ev_max_str=ev_max_str, rv_min_str=rv_min_str, rv_max_str=rv_max_str)

In [ ]:

rows_list = []
for row_index, row_series in correlation_df.sort_values('pearson_r', ascending=False).iterrows():
    left_column = row_series['left_column']
    right_column = row_series['right_column']
    if ('gun' in left_column.lower()) or ('gun' in right_column.lower()):
        rows_list.append(row_series.to_dict())
pd.DataFrame(rows_list).head(20)

In [ ]:

rows_list = []
for row_index, row_series in correlation_df.sort_values('pearson_r', ascending=False).iterrows():
    left_column = row_series['left_column']
    right_column = row_series['right_column']
    if ('gun' in left_column.lower()) or ('gun' in right_column.lower()):
        rows_list.append(row_series.to_dict())
pd.DataFrame(rows_list).head(20)

In [ ]:

#print(['row_series.{}'.format(fn) for fn in dir(row_series) if 'dict' in fn.lower()])

In [11]:

tables_url = 'https://www.thoughtco.com/gun-owners-percentage-of-state-populations-3325153'
tables_df_list = ssu.get_page_tables(tables_url, verbose=True)

[(0, (52, 3))]


In [ ]:

nutty_df = tables_df_list[0].dropna(axis=0, how='all').dropna(axis=1, how='all')
nutty_df.columns = ['Guns_Rank', 'State', 'Guns_Per_Capita', 'Guns_Registered']
nutty_df = nutty_df.iloc[1:]
nutty_df.set_index('State', inplace=True)
us_stats_df = pd.merge(left=us_stats_df, right=nutty_df, left_index=True,
                          right_index=True, suffixes=('_merge', '_nutty'))
s.store_objects(us_stats_df=us_stats_df)

In [ ]:

suicide_df = s.load_csv(csv_name='Suicide Mortality by State',
                        folder_path=s.data_folder).dropna(axis=0, how='all').dropna(axis=1, how='all')
columns_list = ['Suicide_Year', 'district_abbreviation', 'Suicide_Rate', 'Suicide_Deaths']
suicide_df.columns = columns_list + ['URL']
suicide_df = suicide_df[columns_list]
suicide_df.Suicide_Year = suicide_df.Suicide_Year.map(lambda x: int(x))
suicide_df = s.load_csv(csv_name='Suicide Mortality by State',
                        folder_path=s.data_folder).dropna(axis=0, how='all').dropna(axis=1, how='all')
suicide_df

In [ ]:

for year in suicide_df.Suicide_Year.unique():
    mask_series = (suicide_df.Suicide_Year == year)
    df = suicide_df[mask_series]
    columns_list = ['district_abbreviation', 'Suicide_Rate_{}'.format(year),
                    'Suicide_Deaths_{}'.format(year)]
    df.columns = ['Suicide_Year'] + columns_list
    df = df[columns_list]
    us_stats_df = pd.merge(left=us_stats_df, right=df, how='inner', on='district_abbreviation', suffixes=('_merge', '_suicide'))

In [ ]:

print(us_stats_df.columns.tolist())

In [ ]:

import re

for column_name in ['Guns_Rank', 'Guns_Registered']:
    us_stats_df[column_name] = us_stats_df[column_name].map(lambda x: int(x))
us_stats_df.Guns_Per_Capita = us_stats_df.Guns_Per_Capita.map(lambda x: float(x))
for year in [2005, 2014, 2015, 2016, 2017]:
    for infix in ['Rate', 'Deaths']:
        column_name = 'Suicide_{}_{}'.format(infix, year)
        us_stats_df[column_name] = us_stats_df[column_name].map(lambda x: int(re.sub(r'[^0-9\.]+', '', str(x))))

In [ ]:

us_stats_df.Suicide_Deaths_2017.unique()

In [12]:

#us_stats_df['district_abbreviation']
file_path = os.path.join(s.data_folder, 'html', 'us_state_abbreviations.html')
#tables_url = 'https://www.50states.com/abbreviations.htm'
tables_df_list = ssu.get_page_tables(file_path, verbose=True)

[(0, (60, 2))]


In [ ]:

abbrevs_df = tables_df_list[0].dropna(axis=0, how='all').dropna(axis=1, how='all')
abbrevs_df.columns = ['State', 'district_abbreviation']
us_stats_df = pd.merge(left=us_stats_df, right=abbrevs_df, on='district_abbreviation', suffixes=('_merge', '_abbrevs'))
us_stats_df.set_index('State', inplace=True)
us_stats_df.sample(5).T

In [ ]:

s.store_objects(us_stats_df=us_stats_df)

In [ ]:

columns_list = ['Guns_Rank', 'Guns_Per_Capita', 'Guns_Registered', 'Suicide_Rate_2017', 'Suicide_Deaths_2017', 'Suicide_Rate_2016',
                'Suicide_Deaths_2016', 'Suicide_Rate_2015', 'Suicide_Deaths_2015', 'Suicide_Rate_2014', 'Suicide_Deaths_2014',
                'Suicide_Rate_2005', 'Suicide_Deaths_2005']
ssu.get_column_descriptions(df=us_stats_df, column_list=columns_list)

In [ ]:

get_max_rsquared_adj(df=us_stats_df, columns_list=columns_list,
                     verbose=False).sort_values('max_similarity', ascending=False)

In [ ]:

import matplotlib.pyplot as plt

# Use the following only if you are on a high definition device
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina')

import seaborn as sns
import matplotlib.colors as mcolors

basecolor_list = list(mcolors.BASE_COLORS.keys())

In [ ]:

import re
from scipy import stats
%run ../load_magic/dataframes.py

us_stats_df = s.load_object('us_stats_df')
kwargs = dict(textcoords='offset points', ha='left', va='bottom',
              bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
              arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))

ev_column_name = 'Guns_Registered'
column_description_dict[ev_column_name] = 'Number of Guns Registered'
s.store_objects(column_description_dict=column_description_dict)

rv_column_name = 'Suicide_Deaths_2017'
column_description_dict[rv_column_name] = 'Suicide Deaths in 2017'
s.store_objects(column_description_dict=column_description_dict)

ev_min_str = 'least gun-nutty'
ev_max_str = 'most gun-nutty'
rv_min_str = 'least suicidal'
rv_max_str = 'most suicidal'
ev_min_xytext = (-5, 150)
ev_max_xytext = (-130, -100)
rv_min_xytext = (20, -30)
rv_max_xytext = (-100, -50)
az_xytext = (-80, 50)
columns_list = [ev_column_name, rv_column_name]
show_linear_scatterplot(us_stats_df, columns_list, ev_min_str=ev_min_str,
                        ev_max_str=ev_max_str, rv_min_str=rv_min_str, rv_max_str=rv_max_str)

In [ ]:

s.store_objects(us_stats_df=us_stats_df)

In [ ]:

tables_url = 'https://en.wikipedia.org/wiki/List_of_U.S._states_and_territories_by_GDP'
tables_df_list = pd.read_html(tables_url)
print([(i, df.shape) for (i, df) in enumerate(tables_df_list) if df.shape[0] > 50])

In [ ]:

state_gdps_df = tables_df_list[2].dropna(axis=0, how='all').dropna(axis=1, how='all')
state_gdps_df.columns = ['Rank', 'State', '2018', '% of Nation', 'Region']
state_gdps_df = state_gdps_df.iloc[1:]
state_gdps_df.set_index('State', inplace=True)
for column_name in ['Rank', '2018']:
    state_gdps_df[column_name] = state_gdps_df[column_name].map(lambda x: int(str(x).split('[')[0]))
for column_name in ['% of Nation']:
    state_gdps_df[column_name] = state_gdps_df[column_name].map(lambda x: float(str(x).split('[')[0]))
for column_name in ['Region']:
    state_gdps_df[column_name] = state_gdps_df[column_name].map(lambda x: str(x).split('[')[0])
state_gdps_df

In [ ]:

us_stats_df = load_object('us_stats_df')
print(us_stats_df.shape, state_gdps_df.shape)
us_stats_df = pd.merge(left=us_stats_df, right=state_gdps_df, left_index=True, right_index=True, suffixes=('_merge', '_gdp'))
print(us_stats_df.shape)
us_stats_df.columns.tolist()

In [ ]:

us_stats_df.columns = ['White_Percent', 'Black_Percent', 'Hispanic_Percent', 'Asian_Percent', 'Native_Percent', 'Islander_Percent',
                          'Multi_Percent', 'Gini_Index', 'Effectiveness_Rank', 'Health_Care_Score', 'Education_Score', 'Economy_Score',
                          'Infrastructure_Score', 'Opportunity_Score', 'Fiscal_Stability_Score', 'Crime_Corrections_Score',
                          'Natural_Environment_Score', 'district_abbreviation', 'GDP_Rank', 'GDP_2018', 'GDP_Percent', 'State_Region']
store_objects(us_stats_df=us_stats_df)

In [ ]:

us_stats_df.sample(10).T

In [ ]:

import matplotlib.pyplot as plt

# Use the following only if you are on a high definition device
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina')

import seaborn as sns

explanatory_variable = 'Effectiveness Rank'
ev_column_name = 'Effectiveness_Rank'
response_variable = 'GDP Rank'
rv_column_name = 'GDP_Rank'

In [ ]:

df = us_stats_df.copy()
ev_min_str = 'most effective'
ev_max_str = 'least effective'
rv_min_str = 'highest GDP'
rv_max_str = 'lowest GDP'
ev_min_xytext = (-5, 150)
ev_max_xytext = (-135, -30)
rv_min_xytext = (20, -10)
rv_max_xytext = (-100, -50)
us_xytext = (-90, 40)

fig1_fig = plt.figure(figsize=(12,8))
columns_list = [ev_column_name, rv_column_name]
df = df[columns_list].dropna()

# First order (linear) scatterplot
merge_axes_subplot = sns.regplot(x=ev_column_name, y=rv_column_name,
                                 scatter=True, data=df)
xlabel_text = plt.xlabel('{} (explanatory variable)'.format(explanatory_variable))
ylabel_text = plt.ylabel('{} (response variable)'.format(response_variable))
kwargs = dict(textcoords='offset points', ha='left', va='bottom',
              bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
              arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
ev_max = df[ev_column_name].max()
ev_min = df[ev_column_name].min()
rv_min = df[rv_column_name].min()
rv_max = df[rv_column_name].max()
for label, x, y in zip(df.index, df[ev_column_name], df[rv_column_name]):
    if (x == ev_min):
        annotation = plt.annotate('{} ({})'.format(label, ev_min_str), xy=(x, y), xytext=ev_min_xytext, **kwargs)
    elif (x == ev_max):
        annotation = plt.annotate('{} ({})'.format(label, ev_max_str), xy=(x, y), xytext=ev_max_xytext, **kwargs)
    elif (y == rv_min):
        annotation = plt.annotate('{} ({})'.format(label, rv_min_str), xy=(x, y), xytext=rv_min_xytext, **kwargs)
    elif (y == rv_max):
        annotation = plt.annotate('{} ({})'.format(label, rv_max_str), xy=(x, y), xytext=rv_max_xytext, **kwargs)

In [ ]:

tables_url = 'https://en.wikipedia.org/wiki/List_of_U.S._states_by_homicide_rate'
tables_df_list = pd.read_html(tables_url)
print([(i, df.shape) for (i, df) in enumerate(tables_df_list) if df.shape[0] >= 50])

In [ ]:

homicide_df = tables_df_list[0]
homicide_df.set_index('State', inplace=True)
for column_name in homicide_df.columns:
    homicide_df[column_name] = homicide_df[column_name].map(lambda x: float(x))
homicide_df.columns = ['Homicide_Rate_{}'.format(cn) for cn in homicide_df.columns]
homicide_df.sample(5)

In [ ]:

print(us_stats_df.shape, homicide_df.shape)
us_stats_df = pd.merge(left=us_stats_df, right=homicide_df, left_index=True, right_index=True, suffixes=('_merge', '_homicide'))
print(us_stats_df.shape)
us_stats_df.columns.tolist()

In [ ]:

store_objects(us_stats_df=us_stats_df)

In [ ]:

us_stats_df.sample(10).T

In [ ]:

import matplotlib.pyplot as plt

# Use the following only if you are on a high definition device
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina')

import seaborn as sns

explanatory_variable = 'Percent Black'
ev_column_name = 'Black_Percent'
response_variable = 'Homicide Rate 2014'
rv_column_name = 'Homicide_Rate_2014'

In [ ]:

def show_linear_scatterplot(merged_df, columns_list, ev_min_str=None, ev_max_str=None, rv_min_str=None, rv_max_str=None):
    ev_column_name = columns_list[0]
    rv_column_name = columns_list[1]
    explanatory_variable = get_column_description(ev_column_name)
    response_variable = get_column_description(rv_column_name)
    if (ev_min_str is None):
        ev_min_str = 'minimum {}'.format(explanatory_variable)
    if (ev_max_str is None):
        ev_max_str = 'maximum {}'.format(explanatory_variable)
    if (rv_min_str is None):
        rv_min_str = 'minimum {}'.format(response_variable)
    if (rv_max_str is None):
        rv_max_str = 'maximum {}'.format(response_variable)
    
    df = merged_df.copy()
    columns_list = [ev_column_name, rv_column_name]
    df = df[columns_list].dropna()
    ev_max = df[ev_column_name].max()
    ev_min = df[ev_column_name].min()
    rv_min = df[rv_column_name].min()
    rv_max = df[rv_column_name].max()
    ev_max_labeled = False
    ev_min_labeled = False
    rv_min_labeled = False
    rv_max_labeled = False
    
    # First order (linear) scatterplot
    fig1_fig = plt.figure(figsize=(12,8))
    merge_axes_subplot = sns.regplot(x=ev_column_name, y=rv_column_name,
                                     scatter=True, data=df)
    xlabel_text = plt.xlabel('{} (explanatory variable)'.format(explanatory_variable))
    ylabel_text = plt.ylabel('{} (response variable)'.format(response_variable))
    
    # Add annotations
    for label, x, y in zip(df.index, df[ev_column_name], df[rv_column_name]):
        if (x == ev_min):
            if not ev_min_labeled:
                ev_min_labeled = True
                annotation = plt.annotate('{} ({})'.format(label, ev_min_str), xy=(x, y), xytext=ev_min_xytext, **kwargs)
        elif (x == ev_max):
            if not ev_max_labeled:
                ev_max_labeled = True
                annotation = plt.annotate('{} ({})'.format(label, ev_max_str), xy=(x, y), xytext=ev_max_xytext, **kwargs)
        elif (y == rv_min):
            if not rv_min_labeled:
                rv_min_labeled = True
                annotation = plt.annotate('{} ({})'.format(label, rv_min_str), xy=(x, y), xytext=rv_min_xytext, **kwargs)
        elif (y == rv_max):
            if not rv_max_labeled:
                rv_max_labeled = True
                annotation = plt.annotate('{} ({})'.format(label, rv_max_str), xy=(x, y), xytext=rv_max_xytext, **kwargs)
        elif (label == 'Arizona'):
            annotation = plt.annotate('{} (my home state)'.format(label), xy=(x, y), xytext=az_xytext, **kwargs)
    
    # Add r-squared
    x = df[ev_column_name].values
    y = df[rv_column_name].values
    plt.text(0.92, 0.965, r'$r^2 = {0:.2}$'.format(stats.pearsonr(x, y)[0] ** 2), fontsize=20, alpha=0.25,
             horizontalalignment='center', verticalalignment='center', transform=merge_axes_subplot.transAxes)

In [ ]:

import re
from scipy import stats

us_stats_df = load_object('us_stats_df')
kwargs = dict(textcoords='offset points', ha='left', va='bottom',
              bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.5),
              arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
ev_min_str = 'least black'
ev_max_str = 'most black'
rv_min_str = 'least murderous'
rv_max_str = 'most murderous'
ev_min_xytext = (-5, 150)
ev_max_xytext = (-130, -100)
rv_min_xytext = (20, -10)
rv_max_xytext = (-100, -50)
az_xytext = (-80, 60)
columns_list = [ev_column_name, rv_column_name]
show_linear_scatterplot(us_stats_df, columns_list, ev_min_str=ev_min_str, ev_max_str=ev_max_str, rv_min_str=rv_min_str, rv_max_str=rv_max_str)


----

In [ ]:

column_name = 'Total_Gun_Murder_Deaths_2010'
cb1 = c.show_colorbar(column_name)

In [ ]:

from matplotlib.colorbar import ColorbarBase
from matplotlib.pyplot import savefig

savefig??

In [ ]:

print([f'cb1.{fn}' for fn in dir(cb1) if 'fig' in fn.lower()])
print([f'cb1.{fn}' for fn in dir(cb1) if 'ax' in fn.lower()])

In [ ]:

fig, ax = plt.subplots(figsize=(1, 6))
print([f'plt.{fn}' for fn in dir(plt) if 'fig' in fn.lower()])
print([f'plt.{fn}' for fn in dir(plt) if 'ax' in fn.lower()])

In [ ]:

from matplotlib.backends.backend_agg import FigureCanvasAgg as fga

canvas_obj = fga(figure=fig)
print([f'fga.{fn}' for fn in dir(fga) if not fn.startswith('_')])
print([f'canvas_obj.{fn}' for fn in dir(canvas_obj) if not fn.startswith('_')])


----

In [11]:

if 'Estimated_IQ' not in us_stats_df.columns:
    if s.csv_exists('iq_by_us_state'):
        iq_by_us_state_df = s.load_csv('iq_by_us_state')
        iq_by_us_state_df.set_index('State', drop=True, inplace=True)
        iq_by_us_state_df.columns = ['Estimated_IQ', 'Percent_Whites_in_Non_Public_Education', 'Gross_Product', 'Health', 'Violent_Crime',
                                     'Government_Effectiveness']
        us_stats_df = pd.merge(left=us_stats_df, right=iq_by_us_state_df, how='outer', left_index=True,
                               right_index=True, suffixes=('_stats', '_iq'))
        s.store_objects(us_stats_df=us_stats_df)

In [4]:

if notebook_path is not None:
    !start %windir%\explorer.exe "{os.path.abspath(os.path.dirname(notebook_path))}"